# Low-Rank Adaption (LoRA)
This Notebook introduces how to apply low-rank adaptation (LoRA) to your model of choice using [Parameter-Efficient Fine-Tuning (PEFT) library developed by Hugging Face](https://huggingface.co/docs/peft/index). 


### Learning Objectives
1. Apply LoRA to a model
1. Fine-tune on your provided dataset
1. Save your model
1. Conduct inference using the fine-tuned model

In [ ]:
!pip show ipykernel

In [ ]:
!pip install peft==0.4.0

In [ ]:
!pip install accelerate

In [8]:
mkdir cache

We will re-use the same dataset and model from the demo notebook.

In [9]:
mkdir offload

In [10]:
mkdir working

In [ ]:
!pip install datasets transformers

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"
foundation_model = AutoModelForCausalLM.from_pretrained(model_name, token=os.environ['TOKEN_1'], device_map='cuda', torch_dtype=torch.float32)
tokenizer = AutoTokenizer.from_pretrained(model_name, token=os.environ['TOKEN_1'], device_map='auto', torch_dtype=torch.float32)

data = load_dataset("Abirate/english_quotes", cache_dir="./working/cache"+"/datasets")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)
train_sample = data["train"].select(range(50))
display(train_sample) 

/home/kookmin/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.36s/it]


Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 50
})

In [2]:
# TODO
import peft
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=1,  # dimension of the updated matrices
    lora_alpha=4,  # parameter for scaling
    target_modules=[
        "q_proj",
        # "up_proj",
        # "o_proj",
        # "k_proj",
        # "down_proj",
        # "gate_proj",
        # "v_proj"
    ],
    lora_dropout=0.1,  # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM",
)

In [3]:
# TODO
peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 172,032 || all params: 3,212,921,856 || trainable%: 0.00535437859089966
None


In [4]:
# TODO
import transformers
from transformers import TrainingArguments, Trainer
import os

tokenizer.pad_token = tokenizer.eos_token

output_directory = os.path.join("./cache/working", "peft_lab_outputs")
training_args = TrainingArguments(
    report_to="none",
    output_dir=output_directory,
    auto_find_batch_size=True,
    learning_rate= 3e-2, # Higher learning rate than full fine-tuning.
    num_train_epochs=5,
    # no_cuda=True
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_sample,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

Step,Training Loss


Step,Training Loss


Step,Training Loss


TrainOutput(global_step=125, training_loss=7.612712890625, metrics={'train_runtime': 21.2749, 'train_samples_per_second': 11.751, 'train_steps_per_second': 5.875, 'total_flos': 300417880854528.0, 'train_loss': 7.612712890625, 'epoch': 5.0})

In [5]:
import time
import os

time_now = time.time()

peft_model_path = os.path.join(output_directory, f"peft_model_{time_now}")

trainer.model.save_pretrained(peft_model_path)

In [ ]:
# TODO
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        device_map="cuda:1",  # 두 번째 GPU로 할당
        # quantization_config=config,
        token=os.environ['TOKEN_2']
    )
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, device_map="cuda:1", token=os.environ['TOKEN_2'])
tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})  # pad_token 설정
loaded_model = PeftModel.from_pretrained(model, "/home/kookmin/chaewon/LLM_document_summary/results/checkpoint-27534", 
                                        is_trainable=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 1 has a total capacity of 23.69 GiB of which 28.94 MiB is free. Process 2460656 has 12.93 GiB memory in use. Including non-PyTorch memory, this process has 10.72 GiB memory in use. Of the allocated memory 10.47 GiB is allocated by PyTorch, and 1.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
tokenizer("<|eot_id|>")

{'input_ids': [128000, 128009], 'attention_mask': [1, 1]}

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

# 사용자 정의 StoppingCriteria
class StopOnKeyword(StoppingCriteria):
    def __init__(self, stop_words, tokenizer, max_words=500):
        self.stop_words = stop_words
        self.tokenizer = tokenizer
        self.words = 0
        self.max = max_words

    def __call__(self, input_ids, scores):
        generated_text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        self.words += 1
        if self.words > self.max:
            return any(word in generated_text[-1] for word in self.stop_words)
        return False


In [ ]:
import json
with open('dataR.json', 'r') as F:
    d = json.load(F)
d

In [ ]:
# TODO
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(loaded_model.device)
# inputs_raw = '40억 달러 ‘딜’ 주인공 김봉진 우아한형제들 대표태풍 뒤의 고요함이랄까.\n  40억 달러짜리 ‘딜’(거래)을 마친 뒤의 사무실은 조용했다.\n  음식 배달 앱 ‘배달의민족’ 운영사 ㈜우아한형제들의 서울 송파구 방이동 사옥은 비어 있었다.\n  전 직원들에게 연말 특별 휴가를 선물한 김봉진(44) 대표는 혼자 출근해 남은 일을 처리하고 있었다.\n  축하한다는 말을 건네자 “이제 시작인 걸요”라는 답이 돌아왔다.\n  그에게 기업 매각은 단순한 ‘엑시트’(창업 후 지분 매각으로 이익을 실현하는 일)가 아니었다.\n  성공 스토리 뒤에는 환희의 무게만큼 고민이 자리 잡고 있었다.\n  독일계 음식 배달서비스업체 DH(딜리버리 히어로)가 평가한 우아한형제들의 기업가치는 약 4조8000억원.\n  국내 스타트업 M&A 사상 최대 규모다.\n  앱 하나로 평가받은 기업가치가 GS나 현대건설의 시가총액과 맞먹는다.\n  ‘매각’이라는 표현을 썼지만, 창업자인 김 대표가 회사를 떠나는 것은 아니다.\n  오히려 역할이 커졌다.\n  두 회사가 절반씩 출자해 싱가포르에 세우는 합작법인 ‘우아DH아시아’의 책임자로서 아시아 11개국 사업을 총괄하게 된다.\n  DH는 우아한형제들의 투자자 지분 87%를 인수하고, 김 대표 등 경영진이 가진 지분 13%는 DH 본사 지분으로 전환하기로 했다.\n  그렇게 되면 김 대표는 DH 경영진 가운데 개인 최대 주주가 된다.\n   “더 큰 꿈 위해 글로벌 자본 선택” 김 대표의 고민은 ‘민족’이라는 단어에 닿아 있었다.\n  회사가 외국 자본에 넘어가면서 민족 브랜드가 어울리지 않게 됐다는 시선 때문이다.\n  소비자 반응이 긍정적이지만은 않다.\n “겸허하게 받아들인다.\n  DH와는 경쟁 관계이지만 창업 초기부터 지속해서 교류해왔다.\n  그 과정에서 그들이 지닌 ‘글로벌 DNA’에 놀랐다.\n  DH는 홈그라운드 격인 독일 사업마저 네덜란드 기업에 넘기고 글로벌 마케팅을 강화해왔다.\n  그들과 계속 싸울지, 합쳐서 글로벌 무대로 나갈지 마지막까지 고민했다.\n  더 큰 도전을 위한 선택이라고 이해해줬으면 한다.\n ” 국내 상장도 생각해볼 수 있었을 텐데.\n “국내 상장이 이뤄지지 않아 아쉽긴 하다.\n  난들 여의도 거래소에서 멋있게 상장 축하 종을 쳐보고 싶지 않았겠나.\n  그러나 국내 상장이나 매각을 통해 조달할 수 있는 자본은 한계가 있었다.\n  글로벌 무대에서 경쟁하기엔 턱없이 부족한 규모라고 판단했다.\n  향후 3~4년 사업 시뮬레이션을 해 본 결과, 국내 상장으로는 ‘폼’ 한번 잡은 뒤 서서히 죽어갈 수밖에 없다는 결론을 얻었다.\n '
inputs_raw = '2. 계양구 사회복지 자원현황 \n1)보건복지행정 및 서비스 현황 \n2)사회보장예산 현황\n□ 계양구의 사회개발과 관련된 예산변화 추이를 살펴보면 전체적으로 2003년을 기준으로 약 12.98%의 증가추이를 보이고 있음. 사회보장 예산의 경우약 94.91%의 증가를 보였고 보건 및 생활환경의 경우 6.60%의 증가율을 보이고 있음. \n3)보건복지서비스 대상자 현황\n(1)영•유아 인구 동별현황\n⚬ 계양구 전체 인구에 대한 영•유아의 비율은 8.38%이며 전체 인구에 대한 영•유아비율이 가장 높은 곳은 작전서운동으로 10.59% 로 나타났으며 다음으로 계양2동(9.51%), 계양1동(9.29%)순으로 나타났음. 영•유아 인구가 가장 많은 곳은 계양2동이 28,135명중의 14.86%인 4,182명을 차지하고 있음. \n(2)아동•청소년 동별인구현황\n□ 계양구의 전체 인구에 대한 아동의 비율은 29.03%이며 전체인구에 대한아동의 비율이 높은 동은 계양2동(3.94%)이었으며 작전 서운동(3.36%), 효성2동(3.26)순으로 나타났음. \n⚬ 계양구의 전체 인구에 대한 청소년의 비율은 26.01%이며 전체 인구에 대한 청소년의 비율이 높은 동은 계양2동(3.18%), 효성1동(3.01%), 효성2동(2.88%)순으로 나타났음. \n(3)노인 인구 동별 현황\n⚬ 65세 이상 노인은 계양구 전체인구의 5.05%를 차지하고 있음. 인천광역시의 65세 이상 노령인구 비율이 6.87%로 이 보다는 낮은 비율을 보이고 있고, 전체인구에 대한 노인의 비율이 가장 높은 곳은 계양2동(0.67), 효성1동(0.62), 효성2동(0.52), 작전2동(0.52)순으로 나타났음. □ 계양구의 65세 이상 노인 인구는 2000년부터 약 1%~2%의 감소와 증가를 반복하고 있음. \n(4)장애인 인구 현황\n⚬ 계양구 등록장애인은 9,853명이고 6급이 가장 많은 2,243명이고 5급, 3급, 4급, 2급 순으로 나타났음. \n⚬ 장애 종류별로 보면 지체장애가 5,310명으로 가장 많았으며 다음으로 시각(946명), 뇌병변(761명)으로 나타났음. \n(5)국민기초생활수급자 현황\n□ 계양구 전체 인구에서 국민기초생활 수급자가 차지하는 비율은 1.53%인 5,162명이고, 인천광역시 전체 인구의 0.19%인 것으로 나타났음. \n(6)기타 보호대상자 현황\n□ 계양구의 기타 보호대상자 현황을 살펴보면 우선 의료급여지원 대상자는 2,582세대 4,794명이며, 모자가정의 경우, 514세대 1,347명, 부자가정은 82세대 215명, 소년·소녀가정은 76세대 100명의 보호 대상자가 있음. \n4)부문별 복지자원 현황\n(1)보건·의료시설현황\n□ 총 398개소이고 계산4동에 보건소 1개소, 계양1동에 보건진료소 1개 소가 있으며, 종합병원은 작전동에 한림병원 1개소가 있음. \n(2)보육관련 기관현황\n□ 총 246개의 보육시설이 있으며 11,438명을 수용할 수 있으며 그 중 유치원이 34개소, 방과 후 공부방이 9개소가 있음. \n(3)아동, 청소년복지 관련 기관 및 시설현황\n□ 계양구청소년수련관은 청소년의 건전문화 활동을 조성하기 위하여 교육문화, 생활체육, 상담활동, 동아 리 활동 및 예절교육 등의 프로그램을 운영하고 있음. \n⚬ 3개소의 자치문고센터를 운영하고 있으며 2개소의 공부방이 효성2동과 서운동에 각 위치하고 있음. \n(4)노인복지 관련 기관현황\n□ 계양구 노인복지회관 1개소가 있고, 재가노인복지센터와 보호센 터가 각 1개소 운영되고 있음. \n2)일반주민 복지 실태\n□ 일반주민들에게 복지기관 이용경험에 대해 질문한 결과, 주민 자치센터의 이용률은 76.7로 가장 높게 나타나고 있으나, 보육시설 23.4%, 사회복지관 13%를 제외한, 여타 복지기관이 이용률은 매우 낮은 것으로 나타남. \n⚬ 복지기관의 이용경험이 낮은 것은 일반주민이 노숙자 관련기관이나 자활후견기관과 같은 시설을 이용하지 않기 때문에 당연한 것으로 볼 수 도 있으나, 여성 관련기관이나 종합사회복지관 등은 일반인을 대상으로 하는 프로그램을 운영하는 복지 기관임에도 이용경험이 낮은 것은 문제점으로 지적할 수 있음. \n□ 가정문제 심각성 정도에 대한 질문에서는 대부분의 응답자가 가정에 별다른 문제가 없는 것으로 응답함. ⚬ 다만, 부채문제 4.5점, 일자리문제 4.4점은 비교적 다른 항목에 비해 심각성 정도가 높은 것으로 나타남. ⚬ 전반적인 가정문제의 심각성 정도는 경제적 문제가 조금 있지만, 가족 간 불화를 발생시킬만한 정도는 아닌 것으로 판단됨. \n3)계양구 복지 평가\n□ 일반주민을 대상으로 하여 계양구의 복지 및 환경을 평가 하도록 설문조사표를 설계. \n⚬ 조사결과를 살펴보면, 전반적인 만족도는 보통 이상(3.0)으로 나타나고 있으나, 효성동은 문화환경 2.5, 복지서비스 2.6으로 여타 생활환경과 비교하여 열악한 것으로 평가하고 있고, 작전동은 생태환경을 2.6으로 낮게 평가함. \n⚬ 전반적인 만족도는 동별로 살펴보면, 작전서운동과 계양1동이 높게 나타나고 있고, 효성동과 작전동은 낮게 나타나고 있으며, 계양구의 부문별 만족도는 교통환경이 3.5점으로 가장 높게 나타나고 있고, 복지서비스화 문화환경이 2.9점으로 가장 낮게 나타남. \n□ 사회복지기관별 이용만족도 조사는 조사대상 중 이용경험이 있는 응답자만 대답한 결과로서 대부분의 사회복지기관이 높은 만족도를 나타내고 있음. \n• 주민자치센터는 가장 높은 만족도인 3.7점으로 조사되었으며, 자활후견기관, 장애 인관련기관, 노숙자 관련기관은 3.0점으로 상대적으로 낮은 만족도를 보이고 있으나, 응답자수가 매우 적어 평가를 전적으로 반영하기는 어려움이 있음. \n□ 계양구 관내의 복지기관이 계양구민의 삶에 긍정적으로 기여하고 있는가에 대한 질문에서는 보통내지 약간 도움이 되는 정도라고 해석할 수 있는 3.2점으로 조사됨. \n⚬ 동별로 계양구 복지기간의 역할에 대한 평가를 살펴보면, 효성동이 가장 낮은 점수인 2.9점으로 조사되었고, 계양1동이 3.4로 가장 높은 평가를 내리고 있음. \n□ 일반주민의 계양구 전체 복지서비스 만족도를 평가하는 항목 에서는 작전서운동이 가장 높은 평가점수인 3.4점으로 조사되었고, 반면에 가장 낮은 평가점수로 조사된 곳은 효성동(2.9)으로 조사되였음. \n• 현재 일반주민이 평가하는 계양구 전체 복지서비스에 대한 평가는 긍정도 부정도 아닌 상태로서 향후 복지서비스 공급에 대한 계양구의 역할과 책임이 매우 중요할 것으로 판단됨. \n4)복지수요\n□ 계양구의 각 복지 부문 우선순위를 일반주민에게 1순위, 2순위, 3순위를 구분하여 선정하게 한 결과, 1순위만을 살펴볼 경우 저소득층에 대한 복지수요가 가장 높은 것으로 나타남. \n⚬ 보육복지, 아동복지, 청소년복지, 노인복지 등의 부문은 15.0% 내외로 나타나고 있어 주요 복지 욕구로 판단 됨. \n⚬ 각 동별로 복지욕구를 살펴보면, 작전서운동은 저소득층 복지 부문에 많은 수요를 나타내고 있고 계양1동은 청소년 부문 복지수요가 25.0%, 계양2동은 아동부문 복지수요가 26.7%로 나타나고 있음. \n□ 계양구의 복지 부문 중 우선 순위를 3개 선택하게 했을 경우를 살펴보면, 보육복지가 54.9%로 가장 높게 나타났고, 이밖에 아동 및 청소년 복지 부문에 49.7%와 47%로 다른 부문 보다 높게 나타나고 있음\n⚬ 다만, 한 가지 특이한 것은 1순위를 선택하게 하였을 경우, 저소득층부문에 대한 복지수요가 가장 높게 나타났으나, 3순위 까지 선택하게 했을 경우에는 저소득충에 대한 복지수요 는 총 10개 분야 중 7위를 차지하고 있고, 중요도에서도 300% 중 25.1%만 나타나고 있음. \n5)보건복지 관련 실태 및 평가\n□ 계양구 일반주민의 의료비는 월 평균 3만 4천원으로 조사되었고, 대부분의 주민들이 5만원 이하의 의료비를 지출하고 있는 것으로 나타났으나, 12.2%의 주민들은 5만원 이상의 의료비를 사용하고 있는 것으로 조사됨. \n□ 보건소와 관련된 만족도를 묻는 질문에서는 대부분 만족도가 3.5점 이상 4.0점 사이에 분포하고 있기 때문에 만족도가 높은 것으로 평가할 수 있음. \n• 특이한 점은 계양 1동은 거리의 근접성에 대한 만족도가 높은 반면, 효성동은 매우 낮은 것으로 나타나고 있어 실질적 거리의 문제보다는 심리적 거리의 문제가 주요 평가 기준으로 작용하고 있는 것으로 보임. \n□ 계양구 주민은 질병 발생 시 주 치료 방법은 76.2%로 개인 병 •의원을 이용하고 있는 것으로 나타나고 있고 2순위는 17.3%로 나타나고 있는 약국으로 나타남. \n□ 질병 발생 시 치료희망 기관으로서 개인병의원을 61.9%로 가장 선호하고 있고, 다음으로는 대형병원을 34.1%로 들고 있음. \n□ 현재 치료기관과 치료희망기관 차이가 있다면 어떤 이유인지 묻는 질문에서는 차이가 없다는 응답이 25.9%로 가장 많았고 차이가 있는 경우는, 병이 크지 않아서 22.4%, 대기시간이 길어서 21.1%로 조사되었고, 치료비 부담 때문이라는 응답도 19.5%로 나타나고 있음\n제2절 계양구 지역사회복지의 비전 및 방향\n1. 정책설정 방향의 모델\n1)중점 추진 복지분야 선정 방법\n□ 계양구가 향후 중점적으로 추진해야할 복지부문 선정은 아래 와 같은 기준을 적용하여 도출. \n□ 중점추진분야 설정 기준\n⚬ 중점추진분야라 함은 계양구 복지 분야의 우선 추진분야 설정을 뜻하며, 중점주진분야 설정기준은 크게 세 가지 기준을 둠\n⚬ 첫 번째 요소는 주민의 복지수요, 두 번째 요소는 전문가의 계양구 복지실태 및 기반시설에 대한 의견, 세 번째는 계양구의 복지현황 및 실태를 참조함. \n⚬ 전문가의 평가에 계양구 복지시설에 대한 평가까지 포함되어 있으나, 이를 보다 정확하게 파악하고자 현황 및 실태를 보정 요소로 설정. \n⚬ 단 항목별 가중치는 주민의 복지 수요에 보다 많은 비중을 두어 수요자 중심의 복지서비스 체계 구축을 목표로 함. \n2)부문별 중점 추진분야 선정방법\n□ 각 복지 부문별 중점 추진 사항은 중점추진분야 선정 기준과 유사한 방법을 적용\n□ 부문별 중점추진사항 설정 기준\n⚬부문별 중점추진사항은 각 복지 분야별에서 우선 추진해야할 사항을 뜻하며, 이러한 부문별 중점추진사항의 설정기준은 크게 세 가지 기준을 둠\n⚬첫 번째 항목은 복지대상자의 복지수요, 두 번째 항목은 부문 별 복지 공급자와 전문가의 의견, 세 번째는 계양구의 부문별 복지현황 및 실태를 참조함. \n⚬단 항목별 가중치는 주민의 복지 수요에 보다 많은 비 중을 두어 수요자 중심의 복지서비스 체계 구축을 목표로 함. \n3)서비스 수요 추계방법\n□ 중점추진 복지분야 및 부문별 중점추진사항을 선정하기 위해 서는 서비스 대상인구, 서비스 수요량 등의 추계가 필요. \n⚬ 그러나 인구추계나 서비스 수요추계 등은 과학적인 조사 및 전문적인 기술이 필요하며 각 유형별로 접근방법도 다양함. \n⚬ 따라서 이 부분은 전문연구팀의 지원을 받는 것이 바람직하지만 본 보고서에서는 몇 가지 간단한 접근방법을 예시적으로 제시고 이를 통해 서비스 수요를 추정하여 산출함. \n□ 서비스 대상인구 추정\n⚬ 각각의 서비스 유형별로 대상 인구규모를 추정하는 것은 과학적인 조사와 전문적인 추계방식을 적용해야 하나, 이를 목적으로 각 자치구별로 별도의 조사를 실시하는 것은 비효율적임. \n⚬ 또한 계획기간이 4년 단기이기 때문에 장래의 변화추이는 현재 추세와 크게 차이가 없을 것으로 예상됨. 따라서 가능한 기존 자료를 활용하고, 기존 자료가 없는 경우에는 현재의 추세를 적용하여 추정함. \n⚬ 기존 연구자료가 없는 경우에는 현재 해당 대상집단의 출현률을 장래 인구 또는 가구규모에 적용하여 추정함. 적용가능한 자료로는 성별·연령별 인구규모, 모자가정, 소년소녀가정, 기초생활수급대상자 등이 있음. \n□ 1인당 서비스 수요량 결정\n⚬ 계획에 포함되는 서비스 유형이 너무 다양하기 때문에 본 보고서에서 모든 서비스별 수요량 주정방식을 제시하는 데는 한계가 있음. 또한 적정 서비스 수준을 결정하는 방법은 별도의 전문연구가 필요함. \n⚬ 적정 서비스 수준을 기준으로 하는 방법으로는, 특정 서비스에 대하여 적정한 서비스 공급수준을 미리 결정하고, 이를 서비스 대상인구 규모에 적용하여 수요를 추정하는 방법임. 이 방법을 적용하기 위해서는 각 서비스 유형별 적정 공급수준 에 대한 전문적인 연구가 선행되어야 함. \n⚬ 현재 이용률을 적용하는 방법은 서비스 대상인구 집단 가운데 해당 서비스 또는 시설을 현재 이용하는 비율을 적용하는 방법임. \n<예시>장애인주간보호시설 이용수요 = 장애인수 X 현재 장애인중 시설이용률\n⚬ 이용 희망률을 적용하는 방법은 주민욕구조사에서 해당 서비스 및 시설에 대한 이용요구율을 조사하고 이를 해당 인구집단 규모에 적용하는 방법임. \n<예시〉경로식당 수요 = 서비스 대상노인수 X 이용희망률\n⚬ 계획목표를 설정하는 방법은 해당 자치구의 가용자원 수준에 따라 욕구를 100% 충족시키지 못할 경우, 전체 서비스 요구량 또는 서비스 대상인구 규모 가운데 목표연도까지 어느 정도를 충족시켜줄 것인가를 계획목표로 설정하고 이에 따라 서비스 공급량을 연도별로 제시함. \n2. 계양구 사회복지의 비전(Vision)과 방향\n1)계양구 복지 실태 개괄\n□ 계양구의 인구적 특성\n⚬ 계양구의 인구 특징은 14세이하 인구와 15세〜64세 이하의 경제활동 인구가 많고, 65세이상 인구가 적은 인구분포를 보이는 젊은 도시적 특성을 가짐. \n□ 계양구 복지 수요자 인구규모\n⚬ 대표적 복지 수요자인 계양구의 기초생활보장대상가구수는 2002년 이후 계속 증가하는 추세임. \n⚬ 계양구의 2004년 기초생활보장대상자는 소폭의 증가를 보였으나, 인구수대비 기초생활보장대상자 비율은 전국 평균 2.89% 인천광역시 평균 2.25%에 미치지 못하는 1.31%에 불과한 것으로 나타남. \n⚬ 계양구의 2004년 인구수 대비 등록장애인의 비율은 3.86%로 인천광역시 평균 4.67%보다 낮은 비율을 차지함. \n□ 계양구 일반주민의 복지 수요\n⚬ 일반주민의 복지 수요를 살펴보면 우선 순위를 3개 선택하게 했을 경우에, 보육복지가 54.9%로 가장 높게 나타났고, 이밖에 아동 및 청소년 복지 부문에 49.7%와 47.0%로 다른 부문보다 높게 나타나고 있음. \n⚬ 다만, 한 가지 특이한 것은 1순위를 선택하게 하였을 경우, 저소득충부문에 대한 복지수요가 가장 높게 나타났으나, 3순위 까지 선택하게 했을 경우에는 저소득중에 대한 복지수요는 총 10개 분야 중 7위를 차지하고 있고, 중요도에서도 300% 중 25.1만 나타나고 있음. \n□ 전문가 의견\n• 전문가들은 계양구의 복지 우선순위로 저소득층 58.5%, 노인복지 56.9%, 장애인복지 50.08%의 순으로 꼽았음. \n⚬ 다음으로 보육, 아동복지, 여성복지의 순의 빈도를 보였으나, 각 35.4%, 24.6%, 20.0%의 빈도에 불과해 상위의 저소득충, 노인복지, 장애인복지와 상당한 빈도 차이를 보임. \n⚬ 전문가들은 계양구의 사회복지기관 시설 및 여건에 대한 관심 정도 평가에서 노인 3.3점, 장애인 3.2점, 저소득충 3.1의 순으로 평가하였고, 가장 열악한 평가를 받은 부문은 청소년 2.8점, 노숙자 2.7점, 외국인 2.1점 순으로 나타남. \n⚬ 전문가의 계양구 사회복지기관에 대한 평가는 보건소 96.8점, 주민자치센터 95.9점, 장애인관련기관 93.9점의 순으로 평가함\n□ 복지대상별 기관 현황\n⚬ 계양구 복지대상별 기관현황을 살펴보면 아동(보육)복지분야와 여성복지 분야의 시설이 가장 열악한 것으로 나타남. \n⚬ 이에 반해 노인복지 분야와 지역복지 분야는 시설 및 단체가 비교적 양호하게 설치되어 있는 것으로 나타남. \n2)중점추진분야\n□ 중점추진분야는 위에서언급한 바와 같이 전문가 의견과 일반 주민의 복지욕구에 근거하여 설정함. \n⚬ 단, 시설 구비 유무에 따라 복지 분야별 우선순위를 정하는 참고자료로 활용함. \n□ 복지 분야별 순위\n⚬ 일반주민은 보육, 아동, 청소년 등에 많은 복지욕구를 가지고 있는 것으로 나타났고, 전문가 및 복지공급자의 경우에는 노인, 장애인, 저소득층 등에 우선순위를 두어야 한다는 응답이 많았음. \n⚬ 계양구 복지 중점추진분야는 보육, 노인, 아동, 청소년, 장애인, 저소득층, 여성, 노숙자 순임. 다만, 7순위까지 큰 차이가 나타나지 않음으로 위 순위는 자원 경합 시 참고자료로 활용. \n3)계양구 사회복지의 비전(Vision)과 방향\n□ 계양구 지역사회복지계획의 기본 방향은 앞서 다룬 지역현황 진단, 구민(복지대상자)들의 욕구조사, 그리고 지역을 둘러싼 경제·사회적 여건의 변화를 토대로 하되 중앙정부의 「참여 복지 5개년계획」을 반영. \n□ 정책목표는 설정한 계양구의 비전을 실행하기 위한 과제들로서 영역별 사업계회수립에 있어서 구체적으로 수행할 사업을 도출하고, 사업의 우선순위를 정하는 기준이 될 것임. 이 계획에서는 정책목표를 첫째, 수요자 중심의 복지서비스 확충, 둘쨰, 통합적 지역사회복지 인프라 구축, 셋째, 주민참여를 통한 복지계양의 실현 등으로 제시함. \n□ 추진방향은 각 행정목표를 달성하기 위한 방법들을 제시하는 것이며, 이 계획에서의 추진방향은 중앙정부나 광역자치단체가 아닌 자치구 단위에서의 실현가능성을 고려한 정책이 우선 순위라고 할 수 있음. \n제3장 계양구 부문별 복지계획제1절 보육복지\n1. 복지실태 및 현황\n1)조사개요\n□ 조사 설계 내용\n⚬ 영·유아는 총 140명을 조사하였고, 대상은 계양구에 거주하는 만 60개월 이하의 영·유아가 있는 주민을 대상으로 하였음. \n⚬ 표본추출방법은 동별, 성별에 따라 비례층화추출하였고, 조사 방법은 1:1개별 면접조사를 실시하였음. \n□ 응답자특성\n⚬ 주요 특성을 살펴보면, 부의 학력은 대졸이 전체 중 52.9%로 가장 많았고, 부의 직업은 사무·관리직에 종사하는 사람이 67.1%였으며, 모의 직업은 전업주부가 88.9%로 조사되었음. \n⚬ 가구소득은 200~300만원이라고 대답한 비율이 47.7%로 가장 높았고, 300~400만원이 23.6%로 다음을 차지했음. 주택소유형태는 전세가 57.7%로 나타났음. \n2)복지실태 및 현황\n□ 계양구에 거주하는 양육 서비스 수요자들이 응답한 자녀 양육 방식은 아래 표에서 나타나듯이 대부분 직접 양육하는 것으로(77.9%)나타났음. 그 외에\'보육기관 활용\'이 17.1%, \'부모님께 부탁\'하는 경우는 5.0%임. \n⚬ 또한 직접 양육하거나 부모님께 부탁하지 않고 보육기관을 활용하는 경우에 있어, 응답자의 거주지 별로 편차를 나타내고 있어, 거주지별 보육 서비스 욕구가 상이하게 존재할 것으로 예상되며, 이에 적절한 보육 서비스 제공 계획 수립이 요구됨. \n□ 계양구의 보육시설을 이용한 경험이 있느냐는 질문에 응답한 결과를 살펴보면, 응답자 전체 중에서 30.7%가 민간보육시설을 이용하는 것으로 나타나 가장 높은 이용률을 나타내었음. \n2. 복지서비스 평가 및 욕구\n1)복지서비스 평가\n□ 계양구가 제공하는 보육서비스에 대한 전체 응답자의 만족도 는 2.9로 나타나\'보통\'보다 못 미친 다소간 불만족한 것으로 나타남. \n⚬ 또한 응답자의 거주지별 보육 서비스 만족도가 상이하게 나타나, 이에 맞는 적절한 보육 서비스 제공 계획 수립이 요구됨. \n□ 각 보육시설별 만족도는 응답자 전체의 결과로 공동보육시설에 대한 만족도가 4.0으로서\'다소 만족\'하는 수준으로 가장 높게 나타났음. 다음으로는 국•공립보육시설(3.8), 국•공립유치원과 사립유치원(3.7), 민간보육시설(3.6), 가정보육시설(3.3)로 조사됨. \n□ 보육시설을 이용한 경험이 있는 응답자들에게 보육시설의 만족도를 조사한 결과, 전반적으로\'보통\'수준 이상으로 만족하는 것으로 조사됨. \n⚬조사결과, 보육시설의 여러 만족요소 중, 보육비의 적절성과 서비스 이용정보 제공 항목의 만족도가 가장 낮게 나타나, 응답자들의 보육시설 서비스에 경제적·정보접근적 수요가 많은 것으로 판단할 수 있음. \n□ 보육시설 중 국·공립 유치원이나 사립 유치원을 이용한 경험이 있는 응답자를 대상으로 기관 서비스의 항목별 만족도를 조사한 결과, \'보육비의 적절성\'(2.8)을 제외한 대부분의 만족요소들이 전반적으로\'보통\'의 수준 이상으로 만족하는 것으로 조사됨. \n□ 응답자가 계양구의 보육시설을 이용하면서 느끼는 보육서비스의 문제점에 대하여, 1순위는\'보육료 부담\'이 60.0%로 가장 높은 빈도를 보임. \n□ 또한 응답자가 계양구의 보육시설을 이용하면서 느끼는 보육 서비스의 문제점에 대해 1순위와 2순위를 모두 합하여 조사한 결과도\'보육료 부담\'이 가장 많이 나타남(71.4%). \n⚬ 그 다음으로는\'보육교사 질, 친절도\'(60.7%), \'프로그램 다양성\'(42,9%), \'보육상담•정보제공\'(13.6%), \'특수보육시설 부족\'(11.4%)의 순임. \n□ 응답자들에게 계양구 전체 사회복지서비스에 대한 만족도에 대해 물어본 결과, 전체 응답자 평균이 3.2로 나타나\'보통\'수준보다 다소 높은 것으로 나타남 \n2)복지욕구\n□ 자녀의 양육을 부모님께 부탁할 경우 정부가 지원해야할 부분은\'보육료지원\'이 가장 많은 것으로 나타남. (57.1%). \n□ 응답자가 가장 필요로 하는 보육서비스 유형 중 1순위로 필요로 하는 서비스 유형은 종일제보육(37.9%)이 가장 많았으며 그 다음으로 반일제보육(26.4%), 영아전담보육(13.6%), 휴일보육(11.4%), 24시간보육(9.3%), 장애아통합보육(1.4%)순임. \n□ 응답자가 필요로 하는 보육서비스 유형 1순위와 2,3순위를 모두 합하여 조사한 결과, 종일제보육과 반일제보육(65.7%)이 가장 많았으며, 그 다음으로 휴일보육(57.1%), 영아전담보육(47.9%), 24시간보육(45.7%), 장애아통합보육(5.0%), 장애아전담 보육(4.3%), 기타(1.4%)순임. \n□ 보육복지 향상을 위해 증설되어야 할 보육기관에 대해 조사한 결과, 국·공립 보육시설이 가장 많았으며(55.0%), 그 다음으로 는 영아전담 보육시설(21. 4%), 국·공립 유치원(12.9%)순임\n□ 향후 4년간 계양구가 보육복지를 위해 힘써야 할 최우선 역점 사업은 보육의 질 향상(35.7%)이 가장 많았으며 보육시간 연장(26.4%), 보육교사 전문성 확보(19.3%), 보육프로그램 개발(11.4%), 간식비 지원(3.6%), 보육정보센터 운영(2.9%), 보육시설 확충(0.7%》순임. \n3)보건의료 관련 실태 및 평가\n□ 한 달 평균 자녀의 의료비에 소요되는 비용을 조사한 결과, 응답자 전체 평균 2만9천원이며, 응답 분포는 1 ~ 2만원 사이(32.9%)가 가장 많았으며, 1만원 이하(25.7%), 2~3만원(8.6%), 3~5만원(15.7%), 5만원 이상(7.1%)순임. \n□ 자녀가 아플 경우 주로 대처하는 방식을 조사한 결과, \'병·의원으로 간다\'가 가장 많았으며(91.4%), \'약국에서 약을 사 먹인다\'(5.0%), \'대형병원으로 간다\'(2.1%), \'보건소에 간다\'(1.4%)의 순임. \n□ 자녀가 아플 경우 어느 기관에서 치료를 받기 원하느냐는 질문에 대한 응답을 조사한 결과, \'개인 병·의원\'이 가장 많았으며(55.6%), 종합병원(33.3%), 보건소(11.1%)의 순임. \n□ 치료받고 싶은 의료기관과 현재 주로 이용하는 치료방법과 차이가 있다면 무슨 이유때문인지 물어본 결과, \'차이가 없어서\'(35.7%), \'대기시간이 길어서\'(27.9%)가 가장 많았으며, 병이 크지 않아서(22.1%), 치료비가 부담되서(9.3%), 거리가 멀어서(5.0%)의 순임. \n□ 최근 3년 이내에 계양구 보건소를 이용한 경험이 있는 응답자들에게 보건소의 서비스 항목별로 만족도를 평가한 결과, 전반적으로 다소간 만족하는 수준의 응답결과를 나타내었으며, 친절도(3.9), 이용절차(3.9), 의료비(3.8), 대기시간(3.7), 의료서비스의 질(3.5), 거리(3.4), 필요한 진료과목(3.4), 시설·장비(3.3)의 순으로 나타남. \n3. 보육복지계획\n1)계양구 보육복지 사업현황 및 평가\n□ 2006년 계양구 보육복지 사업 현황\n⚬ 계양구의 보육부분에 대한 예산은 어린이집이나 보육시설 등에 투입되는 예산이 비교적 낮은 것으로 나타남. \n□ 보육 복지 부문 평가\n⚬ 계양구는 영•유아 인구비율이 전국 4.98%, 인천광역시 5.05%, \n계양구 5.41%로 높게 나타나고 있으며 유년부양비의 경우도 전국 26.30%, 인천광역시 27.38%, 계양구 31.51%로 높게 나타 났음. \n⚬계양구 복지욕구 조사결과, 민간보육시설보다 국•공립보보육 시설의 만족도가 높지만 보육시설의 부족으로 민간보육시설 올 이용하는 것으로 나타남. \n⚬ 보육기관에 대한 평가에서는 전반적으로 만족하고 있으나 보 육비 부담이 다소 있는 것으로 평가하고 있으며, 다른 기관과 의 연계와 관련한 만족도도 비교적 낮은 것으로 나타남. \n2)중점추진사항\n□ 보육복지부문의 정책목표는 "건강하고 밝게 자라는 계양의 아이들"로 설정하였음. \n⚬ 복지욕구 조사결과 필요한 보육서비스로 종일제보육, 반일제보육, 휴일보육, 영아전담보육, 24시간보육 서비스 순으로 나타나 이를 반영할 필요가 있음. \n⚬ 증설되어야 할 보육기관으로는 국•공립보육시설이 가장 높게 나타나고 있으며, 보육부문 최우선 역점사업으로는 보육의 질 향상, 보육시간 연장 등으로 나타남. \n⚬ 이에 본 계획에서는 보육시설의 증설과 보육프로그램의 다양화, 보육서비스 질의 향상 및 복지서비스가 절대적으로 필요한 취약계층에 대한 보육복지서비스를 강화에 중점을 둠. \n3)보육복지 중점주진사항\n(1)학습 및 놀이 공간 확대\n□ 보육관련 복지 기관 확충\n⚬ 보육시설의 확충\n- 2010년까지 국•공립보육시설을 현행보다 30%이상 확충하여 보육서비스의 질 제고. \n- 복지서비스과 보육지원팀에서 기획하여 시행\n⚬ 특수보육시설(장애아조기교육)확충\n- 신설되는 국•공립보육시설 중 1개소는 심리, 언어, 음악 치료 등 다양한 장애아의 조기교육이 가능하도록 함\n- 복지서비스과 보육지원팀과 장애인 복지팀의 연계를 통해 사업을 기획하고 진행 주관부서는 보육지원팀이며 사업비는 보육사업 운영비를 통해 충당(구비 25%)\n⚬ 지역사회보육상담실(육아지원센터)운영지원 확대 - 지역의 보육과 관련된 육아관련 정보를 효율적으로 제공하고\n보육관련 상담 활동이 전문적으로 수행되어 보육정보센터의 기능을 할 수 있도록 확대 운영. \n- 복지서비스과 보육지원팀에서 기획•감독하고, 지역사회보육상담실(육아지원센터)에서 자체적으로 시행할 수 있도록 하고 사업비 인상분(시비지원 2006년 기준 50%)은 구비에서 충당. \n□ 보육프로그램 강화\n⚬ 24시간 종일보육\n- 핵가족화, 맞벌이부부의 증가로 인하여 부부가 동시에 집을 비울 수밖에 없는 불가피한 상황 발생 시 이를 해결할 수 있는 24시간 종일보육 프로그램 실시. \n- 현재 4개소에서 2010년까지 10개소를 늘리는 것을 목표로 함. \n- 민간보육기관에 이와 같은 프로그램이 개설될 수 있도록 유도하고 국·공립보육시설은 지역별 고려를 통해 연차적으로 이를 시행. \n⚬ 주말보육프로그램\n- 현대 사희에서 맞벌이 부부의 경우 주말을 제외하고는 정상적인 가정생활을 누리기가 어려움. 이에 건강한 가정을 위해서 부부간의 주말생활을 지원할 필요가 있음. 이에 주말보육프로 그램을 실시하여 건강한 가정을 유지할 수 있도록 지원. \n- 주관부서는 복지서비스과 보육지원팀이며, 주말 보육프로그램이 보다 많이 시행될 수 있도록 보육기관 유도, 사업진행예산은 수익자부담원칙을 적용하고 다만 저소득층, 차상위계층 등 요보호대상자의 경우는 보조금 지급. \n□ 부모협동보육시설지원 및 홍보 강화\n⚬ 부모협동보육시설 설립 지원 및 홍보\n- 부모협동보육시설 설립 및 운영과 관련한 정보 제공. \n- 부모협동보육시설의 활성화를 위한 시범시설의 설치• 운영과\n이에 대한 평가를 통해 지속적 발전 지원. \n- 주관부서는 복지서비스과 보육지원팀 이며, 부모협동보육시설이 활성화 될 수 있도록 지원 및 홍보 강화. \n⚬ 품앗이 공동 보육제도 도입\n- 품앗이 공동보육이란, 아파트 등 주거밀집지역 또는 농촌 등 에서 비숫한 또래의 영•유아 또는 아동을 가진 주민이 공동으로 보육 및 교육을 실시하는 제도. \n- 품앗이 공동 보육이 시행될 수 있도록 제도적 정비 및 지원 하고, 부모교사의 양질의 보육 및 교육을 위한 지원 등. \n- 사업기획부서는 복지서비스과 보육지원팀. \n- 사업예산은 보험료, 보조금 등이며 구비로 충당하되 공동보육 시설의 한 형태로 육성될 수 있도록 함. \n(2)영•유아 관리체계 강화\n□ 취약계층 영•유아에 대한 재가서비스 강화\n⚬ 방문의료지원 강화\n- 취약계층 영•유아에 대한 방문 진료 서비스가 매우 필요하며 특히 정신지체 및 장애 부모의 자녀의 경우는 보건의료 대상에 있어서 사각지대에 놓일 상황이 높음으로 이들에 대한 방문보건진료사업이 필요. \n- 사업기획부서는 보건소 방문보건팀으로 하고 연계부서는 복지서비스과 보육지원팀. \n- 사업예산은 보건의료계획과의 연계를 통해 보건소 예산에서 충당. \n⚬ 분유 및 도시락 배달 서비스 대상 확대\n- 자원봉사센터 및 재가복지서비스센터를 중심으로 한 도시락 배달서비스 대상을 확대하여 적용. \n- 사업기획부서는 주민생활 지원과 서비스 연계팀에서 주관하고 집행은 자원봉사센터 및 재가복지서비스센터를 통함. \n□ 보건의료 서비스 강화\n⚬ 건강관리 방법 방문 교육\n- 아이들을 대상으로 구강보건과 관련하여 이 닦는 방법, 손 씻는 방법 등 기초위생과 관련된 사항을 보건소에서 방문교육\n- 사업주관부서는 보건소 건강증진팀 모자보건실에서 담당할 수 있도록 지역보건의료계획에 반영될 수 있도록 복지서비스과 보육지원팀에서 협조 요청. \n⚬ 방문 진료 사업 확대\n- 어린이집 및 유치원에 방문하여 특이사항이 있는 아이들을 대상으로 방문진료를 실시하여 다양한 질병을 조기발견. \n□ 차상위계충 및 모부자 가정에 대한 보육복지서비스 강화\n⚬ 모•부자가정에 대한 후견인 제도 도입\n- 자원봉사자, 노인단독세대, 일반주민 등 희망자를 대상으로 하여 모•부자가정에 대한 "할머니•할아버지", "큰아버지, 큰어머니" 등으로 불릴 수 있는 대부, 대모 제도 도입\n- 사업기획은 복지서비스과 여성아동복지팀에서 주관하고 시행. \n(3)보육서비스 질 제고\n□ 보육관련 복지기관 종사자 전문성 제고\n⚬ 보육교사 교육 제공\n- 보육교사에 대한 전문성을 제고하기 위해 새로운 영•유아 레크레이션, 보육방법 등이 지속적으로 교육될 수 있는 체계 마련. \n- 복지서비스과 보육지원팀에서 기획•집행하고 사업예산은 구비에서 충당. \n□ 보육기관 서비스 강화\n⚬ 평가인증제도 도입\n- 일반적으로 보육교사의 처우가 매우 열악한 이유로 보육교사의 이직률이 높아 보육의 전문성 제고에 걸림돌이 되어왔음. 이와 함께 보육서비스의 질에 대해서도 욕구조사 결과 낮은 평가를 보임. \n- 이에 여성가족부에서 시행하고자 하는 평가인증제도를 도입하여 보육시설의 서비스 질 제고를 도모. \n- 사업주관부서는 복지서비스과 보육지원팀이며 연계기관은 관 내 보육시설. \n⚬ 보육교사 워크숍 개최\n- 년 2회 어린이집 운영자 및 부모님을 대상으로 "버릇 나쁜 아이 고치기", "소심한 아이 활달하게 만들기" 등 보육일선에서 발생할 수 있는 문제와 이에 대한 해결책을 상호 교류할 수 있는 체계 지원. \n- 기존의 체육대회가 워크숍까지 진행될 수 있도록 유도하고 지원금(1인당 10,000원)을 어린이집 운명자들이 워크숍을 시행할 경우 추가하여 구비로 지원\n□ 보육시설 관리 강화\n⚬ 어린이집 및 유치원의 식사 및 간식 등에 대한 지속적 점검\n- 보육시설에서 아이들의 먹거리와 관련하여 권장 식단을 제공하고 아이들에게 공급되는 식단을 지속적으로 모니터링하여 건강식단이 제공될 수 있도록 함. \n- 사업기획 및 집행부서는 복지서비스과 보육지원팀. ⚬ 보육환경 실태 점검\n- 신규 보육시설의 경우 페놀이나 기타 유독한 화학물질에 의해 아이들의 보육환경이 나빠질 수 있으므로 보건소와 연계한 보육시설 보육환경 점검. \n- 사업기획부서는 복지서비스과 보육지원팀으로하고 연계 부서는 보건소 건강증진팀 모자보건실의 협조를 구함. \n4)핵심 추진사업 연차별 계획\n□ 연차별 사업계획\n⚬ 보육복지 분야 추진사업은 보육관련 복지기관 확충, 보육프로그램 강화, 부모협동보육시설지원강화, 취약계층 영•유아에 대한 재가서비스 강화, 보건의료서비스 강화, 차상위계충에 대한 보육복지서비스 강화、보육관련 복지기관 종사자 전문성 제고, 보육시설 관리 강화, 보육기관 평가체계 구축으로 선정 하였음\n제2절 아동 청소년 복지\n1. 아동•청소년 복지실태 및 현황\n1)아동•청소년 복지 일반현황\n(1)아동복지 조사개요\n□ 조사 설계 내용\n⚬ 아동은 총 179명을 조사하였고, 대상은 계양구 관내의 초등학교에 재학 중인 학생과 그 부모로 하였음. \n。표본추출방법은 효성초등학교 3,6학년, 계양초등학교 2,5학년 안산초등학교 1,4학년 학생을 대상으로 집락 표출하였음. \n⚬ 조사방법은 학교 방문 후 선생님을 상대로 설문의 취지 및 설문 시 유의점을 설명한 후, 배포 후 수거하였음. 설문은 아동대상 질문과 학부모 대상 질문으로 구분하여 가정에서 작성토록 하였음. \n□ 응답자 특성\n⚬ 주요 특성을 살펴보면, 부의 학력은 대졸이 전체 중 54.7%로 가장 많았고, 직업은 사무•관리직에 종사하는 사람이 33.5% 였으면, 모의 직업은 전업주부가 57.5%로 조사되었음. \n⚬ 가구소득은 300-400만원이라고 대답한 비율이 25.1%로 가장 높았고, 200-300만원 이하가 24.0%로 다음을 차지했음. 주택 소유형태는 자가가 68.2%로 나타났음. \n(2)청소년복지 조사 개요\n□ 조사 설계 내용\n⚬ 청소년은 총 206명을 조사하였고, 대상은 계양구에 위치하고 있는 서운고, 계산공고, 안남중, 계양중에 재학중인 학생으로 하였음. \n⚬ 표본추출방법은 각 학교별 2학년 1개학급을 무작위로 선정하였고, 조사방법은 집단 면접조사를 실시하였음. \n□ 응답자 특성\n⚬ 주요 특성을 살펴보면, 부의 직업에 대한 질문에서, 서비스•판매직 비율이 31.1%, 사무•관리직 18.5%로 조사되었고, 가정 형편을 묻는 질문에는 "보통"이라고 대답한 비율이 16.9%였고, "어려운 편" 또는 "여유 있는 편"이라고 답한 비율이 각각 13.6%, 4.5%를 차지하였음. \n⚬ 주택소유형태에 관한 질문에는 전체 중 65.6%가 자기집이라고 대답했고, 전세라고 대답한 비율은 16.0%로 나타났음. \n2)아동•청소년 복지실태\n(1)아동복지 실태\n□ 사회복지관의 아동프로그램에 대해서 알고 있는지 여부에 대해서 53.1%가 모르는 것으로 나타났음. \n□ 지역아동센터(공부방)의 경우 응답자의 2/3가 넘는 78.2%가 모른다고 응답하여 공부방에 대한 홍보가 필요한 것으로 생각됨. \n□ 아동전용시설인 놀이시설에 대해서도 60.9%가 모른다고 응답. \n□ 사회복지관 이용경험을 묻는 질문에 76.5%가 이용을 하지 않는 것으로 나타났으며 이용 경험이 있는 아동의 경우 15.1%가\'간적 있고 재밌다\'고 응답함. \n□ 공부방 이용경험에 대해서 93.9%가 이용하지 않는 것으로 나타남. \n□ 상담경험을 묻는 질문의 경우 98.9%가 이용하지 않는 것으로 나타남. \n□ 어린이 놀이시설 이용 경험을 묻는 질문에 대해 64. 2%가 이용하지 않은 것으로 나타났으며 이용 경험자의 경우 도움이 되었다고 19.0%가 웅답. \n□ 아동과 함께 복지기관을 이용한 경험을 묻는 질문에 대하여 사회복지관(79.3%), 지역아동센터(95.5%), 아동전용시설(73.2%)의 순으로 이용하지 않은 것으로 응답해 거의 이용이 없는 것으로 파악됨. \n□ 복지기관을 이용하지 않는 이유로 응답자의 42.3%가 정보의 부족을 꼽았으며 다음 순으로 기관의 근접성이 떨어지기 때문이라고 응답해 아동복지기관의 홍보와 근접성을 확보하는 방안을 강구해야 함. \n□ 자녀와 관련된 고민이 있을 때 주로 누구와 상의를 하는가에 대해서 친구•이웃•직장동료(59.8%)로 가장 많이 응답하였으며 공식적인 통로인 학교교사나 전문가는 응답자의 2.8%로 주변의 아는 사람과 논의하는 형태가 많은 것으로 나타남. \n(2)청소년 복지 실태\n⚬ 청소년수련관의 경우 49.0%가 인지를 하고 있으며 청소년독서실의 경우 49,5%인 117명이 알고 있으며 54.7%가 이를 이용하고 있는 것으로 나타남. \n□ 사회복지관의 청소년프로그램의 경우 다른 세 곳에 비해 인지율(30.1%)과 이용도(19.8%)가 떨어지는 것으로 나타나고 있어 사회복지관의 청소년 프로그램에 대한 홍보가 필요할 것으로 판단됨. \n□ 청소년들이 고민하는 문제로는 학업문제(2.4)와 여가시간 부족(2.7)을 심각하게 고민하는 것으로 나타났으며 나머지 항목에 대해서는 거의 어려움이 없는 것으로 응답함. \n2. 복지서비스 평가 및 욕구\n1)아동•청소년 복지서비스 평가\n(1)아동 복지서비스 평가\n□ 계양구의 생활환경에 대한 평가를 보면, 53.1%가 보통이라고 응답을 하였으며 32.4%가 아동에게 안 좋다라고 평가를 했으며 8.4%만이 좋다라고 평가하였음. 전반적인 생활환경 수준의 경우 보통 이하(2.7)로 평가하는 것을 알 수 있음. \n□ 사회복지관 아동프로그램에 대한 만족도의 경우 51.4%가\'보통\', 24.3%가 불만족, 24.3%가 만족하는 것으로 나타남. \n□ 지역아동센터에 대한 응답자의 75.0%가 보통으로 응답을 하였으나 실질적 이용자 수가 적어 그 평가를 내리기 어려움. \n□ 어린이 전용시설 만족도에 대해서 응답자의 75.5%가\'보통\'이라고 응답. \n□ 아동 복지기관 부분별 서비스 만족도를 살펴보면 사용료(3.2)항목이\'보통\'이상의 평가가 나왔을 뿐 나머지 항목의 경우 전반적인 개선이 요구되고 있음. \n□ 아동복지기관의 아동복지 기여도와 관련하여 아동복지기관이 기여를 못하는 것으로 평가하고 있음(평균 2.5). 이는 사용경험을 통한 평가보다는 아동복지기관의 전반적인 서비스를 받은 경험이 없기 때문에 나타는 문제로 판단됨. \n□ 계양구의 사회복지서비스 만족도는 40.2%가 불만족스럽다는 응답을 하였으며 만족의 경우 1.7%로 매우 낮은 것으로 나타났음. \n(2)청소년 복지서비스 평가\n□ 계양구의 청소년 생활환경에 대한 평가결과를 보면, 전제 응답자의 44.7%가 불량하다고 응답을 하였고, 양호하다고 평가한 응답자는 12.1%에 불과해 계양구의 청소년 생활환경을 전반적으로 안 좋은 것으로 평가하고 있음(평균 2.6). \n□ 청소년들이 생각하는 유해환경으로부터의 안전 수준의 경우 전체 평균 2.4로 응답자들이 전반적으로 안전하지 못하다고 평가 하는 것을 알 수 있음. 웅답자의 55.8%가 불안전하다고 느끼고 있으며 반면에 안전하다고 응답한 비율은 7.8%에 불과함. \n□ 계양구 전체 사회복지서비스에 대해서 청소년들은 평균 2.5로 불만족스러운 것으로 나타났으며 만족한다는 응답은 7. 3%에 불과한 것으로 나타남. \n□ 청소년 관련 시설에 대한 만족도 평가의 경우 전반적으로 보통의 평가가 나왔음. 청소년수련관의 경우 불만족스럽다는 반응을 보였으며 청소년 독서실(3,1), 사회복지관(3.1), 청소년활동단체 및 기관(3.2)등에 대한 만족 수준이 보통임을 알 수 있음. \n2)아동•청소년 복지욕구\n(1)아동복지 욕구\n□ 아동들의 경우 사회복지관의 아동프로그램의 확충이 필요하다고 43.6%가 응답을 하였으며 다음으로 놀이터와 같은 아동전용시설의 확충(23,5%)을 원하는 것으로 나타남. \n□ 부모님은 아동과 관련하여 가장 걱정스러운 문제로 학교공부(29.1%), 건강(16.2%), 사교육비(14. O%)의 순으로 응답함. \n□ 부모님은 아동과 관련하여 가장 걱정스러운 문제에 대해서(1+2순위)를 물어본 결과 사교육비(48.0%), 학교공부(44.1%), 건강(23.5%)의 순으로 응답함. \n□ 양육과 관련하여 가장 필요한 서비스에 대하여 응답자의 40.8%가 자녀교육보조비 지원을 1순위로 꼽았으며 다음으로\'방과 후 공부방 제공\', \'학습지도\'순으로 나타남. \n□ 양육과 관련한 가장 필요한 서비스(1+2)를 선택했을 경우도 자녀 교육보조비 지원(50.3%)을 1순위로 응답하였으며 자녀의 학습지도(41.3%), 방과 후 공부방 제공(33.0%)순으로 나타남. \n□ 방과 후 교실 참가여부를 묻는 질문에 대하여 64.5%가 보낸다고 응답해 방과 후 교실에 대한 수요가 매우 높은 것으로 나타남. \n□ 방과 후 교실에 불참하는 이유에 대해서는 기관의 접근성(29.0%), 프로그램에 대한 정보 미비(24.2%), 필요성이 없다(22.6%)의 순으로 방과 후 교실이 성공적으로 운영되기 위해서는 교통 불편 해소와 방과 후 프로그램의 내실화가 필요할 것으로 판단됨. \n□ 방과 후 교실 참여의사를 밝힌 응답자를 대상으로 방과 후 교실 프로그램 우선순위를 묻는 질문에 대하여 예•복습 숙제지도(41.0%), 취미•예능교실(2 8.2%), 독서•과학탐구 활동(17.9%)순으로 나타났다. \n□ 프로그램 우선순위를(1 + 2순위)로 물었을 경우 취미•예능교실(52.1%), 예•복습 숙제지도(49.6%), 독서•과학탐구 활동(45. 3%)로 대부분의 응답자들이 3개 부문에 대한 필요성이 높은 것으로 나타남, \n□ 가장 필요한 아동 복지 기관을 묻는 질문에 대하여 32.4%가 지역아동센터가 우선적으로 필요하다고 응답을 하였으며 다음으로 아동전용시설(30.2%), 아동복지관(22.9%)이 필요하다고 응답함. \n□ 향후 4년간 계양구가 추진해야 할 역점과제로 응답자의 60.3%가 아동복지 시설의 확충을 꼽았으며 다음으로 방과 후 프로그램 활성화(33. 5%)로 나타남. \n□ 어린이 과학관 설립을 알고 있는지 여부에 대해서 84.9%가\'아니오\'로 응답해 아동관련 시설의 설립과 관련한 홍보가 부족한 것으로 판단됨. \n□ 어린이 과학관을 건립하게 된다면 필요한 프로그램을 묻는 개방형 질문에 과학과 관련된 직접체험 프로그램(17.9%)에 대한 요구가 가장 많은 것으로 나타남. \n(2)청소년 복지 욕구\n□ 청소년복지와 관련하여 노력해야 될 사항으로 청소년들은\'청소년들의 여가•문화 공간 확보를 가장 많이 꼽았으며(53.4%)다음으로 다양한 청소년놀이 문화의 개발(21.8%), 청소년 유해 환경개선(18.4%)에 응답하였음. \n□ 이는 앞의 청소년의 고민을 묻는 질문과 유해환경의 수준을 묻는 내용과 응답이 일치하는 것으로 환경개선과 청소년들의 여가•문화 공간 확보를 위한 계획이 필요할 것으로 판단됨. \n□ 청소년 이용시설•공간의 충분성에 대해서 청소년들은 전반적 으로 불충분(2.5)하다는 평가를 하고 있는 것으로 나타났으며, 모든 항목에 대해서 불충분하다는 평가를 하고 있어 각 시설들에 대한 확보가 필요할 것으로 판단됨. \n□ 청소년이 뽑은 복지수요프로그램 1순위는 청소년들의 자유로운 자치활동이 가능한 프로그램(36.9%)올 뽑았고 다음으로 문화활동의 강화(17.5%), 자기계발 및 전문직업훈련을 위한 프로그램(12.1%)순으로 나타남. \n□ 청소년이 필요로 하는 복지프로그램에 대한 1순위부터 3순위의 합산에서도 앞의 결과와 똑같이 자치활동프로그램, 문화활동 프로그램, 전문교육활동프로그램 순으로 나타남. \n□ 개방형 질문으로 물어 본 건의사항의 경우 독서실•도서관 건립(2.9%), 여가향유시설(2.9%), 청소년 관련 시설의 확충(2.9%)등 다양한 의견을 제시함. \n3)아동의 보건의료 관련 실태 및 평가\n□ 아동이 아플 경우에 이용하는 주 치료 기관으로 병•의 원(89.4%)로 압도적이었으며 보건소를 간다는 옹답은 0.6%로 아동이 아플 경우 보건소를 이용하는 경우가 거의 없는 것으로 나타남. \n□ 자녀가 아플 경우 치료받고자 하는 기관의 경우 개인병원•의원이 58.7%로 가장 많이 응답을 하였으며 보건소는 7.8%로 보건소를 이용할 의사가 낮은 것으로 나타남. \n□ 자녀의 치료희망기관과 현재 치료하는 기관의 불일치 이유를 묻는 질문에 대하여 응답자의 41.9%가\'병이 크지 않아서\'라고 응답을 하였지만 그 다음이\'거리가 너무 멀어서(24.0%)\'로 나타나 의료기관에 대한 근접성을 고려할 필요 있음. \n□ 월 평균 자녀를 위해 사용하는 의료비의 경우, 응답자의 36.4%가 1만원 이하로 나타났으며 평균적으로 2만7천9백원이 자녀의 의료비로 사용하는 것으로 나타남. \n3. 아동•청소년 복지계획\n1)계양구 아동•청소년복지 사업현황 및 평가\n□ 2006년 계양구 아동•청소년 복지 사업 현황\n⚬ 계양구는 아동•청소년 인구비율이 아동(10-14세, 괄호는 5-9세)의 경우 전국 7.31(6.69), 인천광역시 8.02(6.95), 계양구 9.42(7.92)이고 로 높고, 청소년의 비율(15세-19세)도, 6.42%, 6.91%, 7.68%로 높음. \n⚬ 이에 지역사회복지계획 수립에 있어, 지역적 특성을 고려하여 반영해야할 복지 부문임. \n⚬ 그러나 계양구의 사업현황을 살펴보면, 다른 복지부문에 비해 예산투입이 비교적 적은 것을 알 수 있음. \n□ 아동•청소년 복지부문 평가\n• 아동은 복지기관에 대한 인식이 매우 낮으며, 청소년은 인식 정도는 양호한 편이나, 사용면에서는 활성화 되었다고 평가하기 어려움. \n⚬ 청소년의 경우, 청소년 유해환경에 대해 매우 우려할 만한 평가를 하고 있으며, 고민과 관련해서는 학업문제와 자유시간 부족을 꼽았고, 복지 서비스에 있어, 주요역점사업으로는 청 소년 자치활동 강화로 나타나고 있음. \n⚬ 이에 본 계획에서는 기존의 취약계층에 대한 복지서비스를 유지하는 동시에 아동 및 청소년의 놀이 공간 확충 및 학습 지원을 강화하는 방향으로 설정. \n2)아동•청소년 복지 정책목표\n□ 아동•청소년복지 정책목표는 "즐기면서 공부하는 계양의 미래"로 설정함. \n⚬ 욕구조사결과 아동의 경우 지역아동센터의 이용경험이 전체의 4.5%로 매우 낮은 비율로 나타나고 있고 청소년의 경우도 사회복지기관 청소년활동단체 등의 인지도 및 이용도가 낮은 것으로 나타나 복지기관에 대한 정보 제공이 필요함. \n⚬ 아동•청소년은 학습지원에 대한 욕구가 높았으며, 특히 청소년의 경우 문화활동, 자기계발, 직업전문교육 등의 욕구가 높게 나타났음. \n⚬ 이에 본 계획에서는 아동•청소년들의 학습지원활동, 여가지원 활동, 복지기관 활용을 위한 정보제공, 취약계층에 대한 지원 사업을 중점적으로 추진함. \n3)아동•청소년복지 중점주진사항\n(1)놀이와 학습이 함께하는 아동 교육환경 제공\n□ 놀이 공간 확충\n。어린이놀이터 확충\n- 근린공원의 어린이 놀이시설의 놀이기구의 안전성 확보 및 다양화 사업을 실시하고 연차적으로 근린 어린이놀이시설 3개소 확충. \n- 사업기획부서는 도시정비과에서 주관하며, 수요 및 설치장소는 복지서비스과 여성•아동복지팀과 협의. \n- 사업예산은 공원관리(2231:예산과목)시설비에서 충당. \n⚬ 방과 후 교실에 체육 및 놀이활동 강화\n- 방과 후 교실 운영에 있어 체육 및 놀이활동을 강화하여 아이들의 특기 및 소질을 개발. \n- 현재 계양구의 방과 후 교실은 총 5개소가 운영 중이고 사업 예산은 확대 없이 기존 시설의 프로그램 강화. - 사업주관은 복지서비스과 보육지원팀에서 담당. \n□ 교육 및 복지정보 제공\n⚬ 지역아동지원센터 운영 강화\n- 지역아동지원센터의 운영을 활성화하여 아동의 교육 및 복지 서비스 강화. \n- 사업주관부서는 복지서비스과 여성•아동복지팀이며 사업집행은 지역아동센터에 위탁하여 운영. \n- 사업예산은 민간경상보조비(구비 약25%)의 확보를 통함. \n⚬ 교육상담프로그램 강화\n- 부모님이나 아동들이 성장 및 학습과 관련한 문제 발생 시 마땅한 상담대상이 없어 주변 사람에 의존하므로 이에 지역아동 지원센터내에 아동상담실을 1개소 신설하여 시범운영. \n- 사업주관부서는 복지서비스과 여성•아동복지팀이며 사업집행은 지역아동센터에 위탁하여 운영. \n- 사업예산은 민간경상보조비(구비 약25%)의 확보를 통함\n□ 학습지원활동 강화\n⚬ 도•농간 가정 결연사업 실시\n- 도시•농촌 가정의 결연사업을 후원하여 도•농간 상호교류 및 도시 학생들의 생태체험이 가능하도록 지원. \n- 사업주관부서는 주민생활지원과 복지총괄기획에서 기획하여 집행. \n- 도•농 가정 결연사업은 주민자치센터의 협조를 받아 시행하고 본 사업은 비예산 사업임. \n⚬ 방과 후 교실 프로그램 욕구조사\n- 방과 후 교실 프로그램에 대한 정보의 부족으로 참여가 낮은 것으로 나타나고 있으며 방과 후 프로그램으로 1 순위는 취미• 예능교실, 2순위는 예•복습 지도로 나타나 취미와 학습지원에 대한 욕구가 중첩되어 나타나고 있어 정확한 수요조사 필요. \n- 사업주관부서는 주민생활지원과 복지서비스과 여성•아동복지 팀에서 기획하고, 방과 후 교실 이용•비이용자 대상으로 조사 실시. \n□ 취약계층 지원 강화\n⚬ 저소득충 자녀 및 소년•소녀가장 학습 지원강화\n- 대졸 미취업생에 대한 공공근로인력 또는 고학력 공익요원, 자원봉사자 등으로 구성된 "학습도우미"를 운영하여 저소득 층 자녀 및 소년•소녀가장의 학습 지원. \n- 사업주관부서는 복지서비스과 여성• 아동복지팀이고 공공근로 인력 및 공익요원 활용과 관련된 부분은 지역경제과 및 자치 행정과와 협의. \n⚬ 결식아동지원 확대\n- 급식아동 대상자 및 지원규모를 확대하고 지역아동센터 활용 및 귀가 후 도시락 지원 사업 강화. \n- 사업주관부서는 복지서비스과 여성•아동복지팀이며, 사업집행은 지역아동센터 및 재가서비스센터, 자원봉사센터 등과 협조하고, 사업예산은 시비와 구비로 충당(약75:25)되며, 2006년 예산 279,090,000원의 약 10%씩 증액. \n(2)청소년들의 학습의 권리 및 여가활동지원\n□ 여가활동 시설 확충\no X- sports 시설 신설\n- 청소년들의 건전한 여가활동을 지원하기 위해 자전거, 인라인 스케이트 등을 즐길 수 있는 X- sports 시설을 신설하고, 사업 주관부서는 문화생활과 청소년 팀이며, 사업예산은 체육진홍 시설비에서 충당. \no 농촌지역 농구대 확충\n- 도시지역은 상대적으로 공원, 학교시설의 밀도가 높아 농구대가 많이 설치되어 있지만 농촌 지역의 경우 공간은 확보되어 있는 반면 체육시설이 부족하므로 농구대 확충을 통해 이를 해소. \n- 사업주관부서는 문화생활과 청소년 팀. \no 청소년 봉사활동 프로그램의 다양화\n- 실질적인 봉사활동을 통해 봉사의 기쁨을 청소년들이 알게 할 수 있도록 복지시설과 연계한 다양한 프로그램 개발. \n- 사업기획 및 관리부서는 주민생활지원과 서비스 연계팀이며, 자원봉사센터가 주관이 되어 집행. \n□ 문화•학습프로그램 강화o 문화체험활동 강화\n- 청소년복지관 및 복지사업을 활용한 다양한 형태의 문화체험을 할 수 있도록 지원. \n- 사업기획부서는 주민생활지원과 서비스연계팀이며, 연계부서는 문화생활과 청소년 팀. \no 방문직업상담 강화\n- 중•고등학교에 전문가가 방문하여 청소년들의 적성검사 실시 및 적성검사 결과를 바탕으로 한 직업 및 진학 컨설팅. \n- 사업주관부서는 복지서비스과 여성•아동복지팀이며, 연계부서는 문화생활과 청소년 팀 및 교육청이고, 사업예산은 교육청의 예산이 활용될 수 있도록 함\n□ 취약계층 지원 강화\n⚬ 저소득충 자녀 및 소년•소녀가장 학습 지원강화\n- 대졸 미취업생에 대한 공공근로인력 또는 고학력 공익요원, 자원봉사자 등으로 구성된 "학습도우미"를 운영하여 저소득층 자녀 및 소년•소녀가장의 학습 지원. \n- 사업기획 및 관리부서는 문화생활과 청소년 팀이며, 연계부서는 자원봉사센터 및 자치행정과(공익요원)등과 협의. \n- 사업예산은 청소년 공부방운영비(국+시비:구비=50:50,2122)의 확충을 통해 조달. \n。저소득층 자녀를 위한 학습공간 지원\n- 관내 저소득층 자녀들의 학습을 위한 공부방을 50석 규모의 공부방을 연차적으로 3개소 중설. \n- 사업주관부서는 복지서비스과 여성•아동복지팀이며, 사업예산은 민간경상보조(2316)저소득층학생 수능공부방 지원의 확대를 통하여 조달하고 문화공보과 청소년팀과 연계. \n4)핵심 추진사업 연차별 계획\n□ 연차별 사업계획\n⚬ 아동•청소년분야 추진사업은 놀이공간 확충, 여가활동 시설 확충, 교육 및 복지정보 제공, 학습지원활동 강화, 취약계층 지원강화로 선정하였음. \n제3절 노인복지\n1. 노인복지실태 및 현황\n1)조사개요\n□ 조사 설계 내용\n⚬ 노인은 총 150명을 조사하였고, 대상은 계양구 관내의 60세 이상 어르신을 대상으로 하였음. \n⚬ 표본추출방법은 연령별, 동별, 성별에 따라 비례층화 추출하였고, 조사방법은 1:1 가구 방문 면접조사를 실시하였음. \n□ 응답자 특성\n⚬ 주요 특성을 살펴보면, 가구원수가 2명이하라는 대답이 50.0%로 나타났고, 본인의 월평균 소득도 "없다"라고 대답한 비율이 72.0%로 조사됨. \n⚬ 계양구거주기간은 전체 중 58.7%가 10년이상 거주하고 있다고 대답하였고, 복지수혜여부에 대해서는 "해당 없음"이라고 답한 비율이 96.7%로 조사됨\n2)노인복지실태\n□ 계양구 노인의 생활비 충당 방법을 살펴본 결과 자녀도움 65.5%, 저축•연금 13.8%, 자영업 10.3%, 배우자 소득 6.9%, 공공근로 등 3.4%의 순으로 나타남. \n⚬ 노인의 경우 대부분이 정기적인 소득이 없는 상태에서 생활하고 있는 것으로 평가할 수 있음. \n⚬ 조사결과 사회적 일자리의(노인 일자리 사업)경우 작전서운동, 계양1동, 계양2동 거주 노인의 비율이 높게 나타났으며, 계산동, 효성동은 비교적 낮은 비율로 나타나고 있음. \n□ 노인정, 노인대학, 노인복지회관, 주간보호서비스, 가정봉사파견 서비스, 노인공동작업장, 고령자취업알선센터, 시니어 클럽 등 노인복지 관련 시설 이용 경험은 보건소 67.4%, 노인정 56.7%, 노인복지회관 10.7%, 노인대학 10.7%로 나타났고 여타의 시설은 1%내외의 응답자가 이용경험이 있는 것으로 나타나 계양구 노인관련 시설 중 보건소와 노인정이 주된 이용시설임. \n□ 노인복지회관은 작전동의 이용비율이 타 동의 이용비율보다 많게는 4배 이상의 비율을 보이고 있어 노인복지회관의 이용의 장애요인으로 지리•교통 요인이 있는 것으로 판단됨. \n□ 노인관련 시설을 이용하지 않는 이유로는 이용할 필요가 없어서가 58.0%, 기관에 대한 정보를 몰라서가 24.0%, 시설의 위치가 너무 멀어서 5.3%, 시설이 없어서가 2.7%로 나타남. \n⚬ 노인관련 시설을 이용하지 않는 주된 이유가 필요성 및 정보 부재로 나타나, 관련 시설의 홍보가 부족한 것으로 판단되므로 적극적인 기관홍보가 필요함. \n2. 복지서비스 평가 및 욕구\n1)노인복지서비스 평가\n□ 계양구의 복지 및 생활환경 평가결과를 살펴보면 상대적으로 교통환경, 주거환경, 생태환경, 보건서비스를 높게 평가하고 있는 것으로 나타남. \n⚬ 전반적인 평가점수가 3.1 이상을 유지하고 있어 복지 및 생활 환경 평가에서 긍정적인 평가를 유지하고 있음. \n□ 다만, 지리•교통적 항목의 만족도가 복지서비스와 문화환경 등의 평가점수보다 월등히 높게 나타나고 있어, 하드웨어 구성은 충분하나 이를 활용한 복지 및 문화 서비스 등의 소프트웨어가 부족한 것으로 보임\n□ 노인복지분야에 대한 계양구의 관심정도를 평가한 결과 전체 응답자의 80%가 보통이상이라고 평가함. \n□ 노인관련시설을 이용한 경험이 있는 응답자에게 시설 이용 후 가지게 되는 만족도를 조사한 결과, 상대적으로 가정봉사파견, 노인정, 노인복지회관, 보건소, 노인대학의 만족도가 3.6이상으로\'다소간 만족\'하는 것으로 나타남. \n⚬ 대부분의 노인관련시설의 이용자 만족도가 3.0 이상으로\'보통\'의 수준을 기준으로 긍정적인 평가를 나타내었으나, 취업 알선센터, 시니어클럽(노인일자리)의 경우, 2.5,2.0으로 만족 하지 못하는 것으로 나타났음. 이 두 기관의 경우, 이용률도 역시 낮아, 적절한 서비스의 제공 및 기관의 적극적인 홍보 노력이 필요함. \n□ 계양구의 노인사회복지기관들이 노인의 복지 향상을 위해 어느 정도 기여하고 있느냐는 질문에 대한 응답결과는, 전체 평균 3.0으로\'보통\'이라고 응답함. \n2)노인복지욕구\n□ 노인들의 생활에서 생길 수 있는 여러 문제들을 얼마나 심각하게 생각하느냐는 질문에 대한 응답으로, 응답자 대부분이\'보통\'이라고 옹답한 건강, 경제 상태 문제 외에는 다소간 양호하다고 느끼는 것으로 나타남. \n□ 계양구의 노인복지 향상을 위해 가장 우선적으로(1순위)필요하다고 생각하는 기관은 노인복지관이라는 응답이 가장 많았으며(28.0%), 다음으로는 노인정(24.0%), 노인전문병원(16.0%), 경로식당(12.0%)등이었음. \n⚬ 이는 계양구 거주 노인들의 복지욕구가 여가생활, 건강문제에서 많이 발생하고 있음을 나타냄. \n⚬ 또한 노인복지기관의 수요를 응답자의 거주지별로 살펴보면, 대부분의 경우 노인복지관이 가장 필요하다고 응답하였으나, 효성동은\'노인복지관\', 계양2동은\'노인전문병원\'의 수요가 더 높아, 복지기관의 수요에 있어서 복지대상의 접근성을 고려한 입지선택이 요구됨. \n□ 계양구의 노인복지 향상을 위해 필요하다고 생각하는 기관에 대한 순위 응답결과를 종합하여 보면, 노인전문병원이 가장 많았으며, 다음으로는 노인복지관, 무료건강진단 등의 순으로 나타남. \n⚬ 이는 1순위에 가장 많았던 응답결과(노인복지관)와 달리, 노인전문병원, 무료건강진단이 높게 나타나, 계거주 노인의 전반적인 복지욕구가 건강문제에서 가장 많이 발생하고 있음을 나타냄. \n□ 향후 4년간 계양구가 노인복지를 위해 우선적으로 해야 할 역점사업에 대한 응답자들의 1순위 선택을 조사한 결과, 의료보건사업(31.3%)이 가장 많았으며, 소득지원사업(30.7%), 복지시설•주거시설 확충사업(12.7%), 일자리 제공• 취업알선 사업(6.0%), 노인취업교육(1.3%), 노인학대예방센터(0.7)의 순임. \n⚬ 이는 계양구 거주 노인들의 복지욕구가 건강문제, 경제문제에서 많이 발생하고 있음을 나타냄. \n□ 향후 4년간 계양구가 노인복지를 위해 우선적으로 해야 할 역점사업에 대한 응답자들의 1,2,3순위 선택을 종합해 보면, 의료•보건사업, 노인복지시설 확충사업, 소득지원사업(생계비지원)등의 순서임. \n⚬ 이는 1순위에 가장 많았던 응답결과처럼 의료•보건사업, 소득지원사업이 높게 나타나, 계양구 거주 노인의 전반적인 복지욕구가 건강문제 및 경제문제에서 가장 많이 발생하고 있음을 나타냄. 또한 노인복지시설에 대한 수요도 높은 것으로 보임. \n3)보건의료 관련 실태 및 평가\n□ 응답자들이 아플 때의 대처방법으로는 주로 병 • 의원을 이용 한다고 응답하였으며(77.3%), 약국에 가거나(10.0%), 보건소를 이용하는(7.3%)등의 응답결과를 보임. \n□ 보건소를 이용한 경험이 있는 응답자가 느낀 불편 사항으로는, 원하는 진료 분야가 없다는 점이 가장 많은 불편 사항으로 응답되었으며, 거리가 멀다, 의료시설이 노후하다, 보건소로 가기 위해선 도우미가 필요하다는 등의 순서로 나타남. \n□ 응답 노인들이 치료받기 원하는 의료기관으로는, 개인 병•의원(64.0%)이 가장 많았고, 그 다음으로 종합병원(30. 7%), 보건소(5.3%)의 순임. \n□ 응답자의 73.9%가 회망하는 의료기관과 현재 주로 이용하는 치료방법에 차이가 있다고 응답하였으며, 그 이유는\'치료비가 부담 된다\'라고 응답한 결과(32.0%)가 가장 많았음. 그 다음으로 대기시간이 길어서, 거리가 멀어서, 병이 크지 않아서의 순으로 나타남. \n⚬ 이는 계양구 주거 노인의 보건복지욕구가 경제문제에서 많이 발생하고 있다고 볼 수 있음. \n□ 계양구 노인들의 월 평균 의료비는 응답자 전체 평균 3만7천1백원이며, 응답 분포는 1~2만원사이의 응답(30.7%)이 가장 많았으며, 1만원 이하(24.0%), 5만원 이상(17.3%), 2~3만원(15.3%), 3~5만원(12.7%)의 순으로 나타남. \n3. 노인복지계획\n1)계양구 노인복지사업현황 및 평가\n□ 2006년 계양구 노인 복지 사업 현황\n⚬ 계양구는 노인여가복지시설, 가정봉사원파견기관, 노인주간, 단기보호센터 등 노인복지 관련기관이 설치되어 있으나 욕구 조사결과 나타난 노인의료기관의 수요를 충족시킬 수 있는 노인 의료복지 시설이나 노인전문치료시설은 없음. \n⚬ 욕구조사 결과를 살펴보면 경로당에 대한 평가는 매우 긍정적이나 노인복지기관 수요를 살펴보면 노인전문병원, 노인복지회관, 무료건강진단 등의 수요가 높은 것으로 나타남. \n⚬ 앞으로 계양구가 노인복지를 위해서 우선적으로 해야 할 역점사업으로는 의료보건사업, 소득지원사업, 복지시설•주거시설 확충업, 일자리 제공•취업알선, 노인취업교육 등의 욕구가 높게 나타남. \n□ 노인 복지부문 평가\n⚬ 계양구의 노인복지의 경우, 여가시설이나 일반복지부분은 현재 상태에서 일정부분 욕구를 해소할 수 있으나 의료부분과 취업에 대한 복지욕구는 해소 못하는 것으로 평가할 수 있음. \n⚬ 이에 본 계획에서는 기존의 복지프로그램 및 유지 강화시키는 방향으로 설정하고 의료부분과 노인 취업부분을 신설 및 보강하는 방향으로 추진함. \n2)노인 복지 정책목표\n□ 노인 복지 정책목표는 "건강한 삶을 영위하는 어른신들의 계양"으로 설정함. \n⚬ 욕구조사결과 어르신들은 일상생활에 있어서 별 다른 문제가 없는 것으로 조사되었으나 건강문제, 경제상태, 고독(외로움)등이 주요 문제로 파악되었음. \n⚬ 이에 본 계획에서는 취업을 통한 사회참여를 통해 어르신들의 경제문제 및 외로움 문제를 해결하고 의료지원 서비스체계를 강화하여 건강한 삶을 영위할 수 있도록 계획. \n□ 계양구 노인 인구(65세 이상)는 전국 8.87%, 인천광역시 6.87% 에 비해 5.05%로 낮은 비율이고, 전반적으로 감소추세에 있어 이 부분에 대한 적절한 고려가 필요함. \n⚬ 본 계획에서는 이러한 노인복지 욕구의 양적크기를 고려하여 기타 계획 및 현 노인복지기관 및 제도를 내실화하는 방향으로 계획을 수립함. \n⚬ 노인의 경우 저소득층과 중산층 이상이 삻의 질이 극명하게 구분되므로 전체 노인에 대한 일반적 사회복지서비스와 저소득층 노인에 대한 복지서비스간의 차별화하여 계획을 수립. \n3)노인 복지 중점추진사항\n(1)어르신의 삶의 질 제고\n□ 노인복지기관 확충\n⚬ 노인문화회관(중대형 경로당)건립\n- 현재 노인복지관의 지역적인 문제로 인하여 일부지역(효성동, 계양1동)의 어르신이 노인복지관에서 제공하는 서비스를 받기 어려워, 노인복지관에 대한 욕구가 많음. 이에 노인복지관의 기능을 할 수 있는 노인문화회관(중대형경로당)을 지역적 입지조건을 고려하여 건립. \n- 사업주관부서는 복지서비스과 노인복지팀이며, 예산은 전액 시비이며, 총 1,657,000,000원의 건립비 소요될 것으로 추정. \n⚬ 경로당 환경개선사업 지속 추진\n- 경로당 운영 및 시설개선사업의 지속적 추진을 통해 어신들의 여가활동 지원 강화. \n- 사업주관부서는 복지서비스과 노인복지팀이며, 사업예산은 경로당개보수비, 경로당 생활집기비 등을 탄력적으로 활용. \n□ 문화(전생)교육 강화\n⚬ 복지관을 이용한 다양한 문화교육 프로그램 개설\n- 노인복지관 및 종합사회복지관에 어르신들의 문화 및 교육욕구를 해소하기 위한 어르신 대상 문화프로그램 개설. \n- 사업기획부서는 복지서비스과 노인복지팀이며, 노인복지회관 에 위탁하여 사업 시행. \n- 사업예산은 기존의 프로그램을 복지욕구에 부응하기 위해 변경 및 신설하는 것이므로 추가 예산 수요 대상이 아님. \n⚬ 관내의 대학 및 기타 교육관련 기관과 연계한 노인대학 개설\n- 사업기획은 복지서비스과 노인복지팀이며, 예산소요는 정원50명에 대해 50% 보조금(30,000원)을 지급할 경우, 연간 1200 만원의 예산 소요. \n- 신규예산이므로 노인복지사업에 반영해야 하고 노인교육사업의 시비 50% : 구비50%로 충당\n□ 노인일자리 사업 확대\n⚬ 재가노인 식사 배달 등(복지형 노인일자리 사업)확대\n- 어르신들의 재가서비스 중 식사배달과 같이 다소 간단한 일이나, 서비스 수혜자들이 심리적 위축이 되지 않는 서비스 등은 건강한 어르신들이 봉사할 수 있도록 기회 제공. \n- 사업주관부서는 복지서비스과 노인복지팀이며, 2010년까지 300개의 일자리 확보. \n- 사업예산은 전액 국비 및 시비로서 연차적으로 확대하여 2010년에 600,000,000원까지 확보. \n⚬ 교육형 및 시장형 노인일자리(사업 확대)\n- 예절학교, 서당 등 어르신들이 학생들에게 제공 가능한 강좌를 개설하여 어르신들의 지혜가 보다 효과적으로 지역사회에 활용될 수 있도록 하고, 현재 43개에서 60개까지 확대. \n- 지하철 택배 사업과 같은 시장에서 이윤을 창출할 수 있는 일 자리를 발굴 보급하여 현재 50개 일자리에서 100개까지 확대. \n- 예산은 전액 국비 및 시비로 지원받을 수 있도록 하고, 사업 주관부서는 복지서비스과 노인복지팀. \n(2)의료지원체계 강화\n□ 재가노인 의료 서비스 강화\n⚬ 보건소 방문보건팀 확충\n- 사업주관부서는 보건소 방문보건팀이나 현재 보건소의 방문보건팀은 총10명으로 구성되어 있으나, 수요를 고려할 경우 인력의 부족으로 충분한 서비스 제공이 되고 있다고 판단하기 어려움. \n- 따라서 보건소 및 지역보건의료계획에 이를 반영하여 재가의료서비스의 양•질적 수준 제고하기 위해서는 간호사 인력 및 자원봉사 의사인력의 확충이 매우 필요함, 사업예산은 인건비 에서 충당. \n⚬ 독거노인 안부 묻기 전화 프로그램 실시\n- 독거노인들에게 정기적으로 전화를 걸어 건강상의 악화 혹은 진료가 필요 시 이를 조기에 발견하여 관계기관과 협의하여 독거노인들을 위한 의료서비스 제공. \n- 건강뿐만 아니라 일상적인 말벗의 기능도 할 수 있도록 독거 노인별 담당자를 정하고 최소통화시간 및 횟수를 정해 이에 따라 운영. \n사업주관은 복지서비스과 노인복지 팀이며, 복지형 노인일자리 사업 프로그램으로 활용. \n□ 노인의료기관 확충\n• 노인전문요양시설건립\n- 복지욕구조사 결과를 살펴보면, 노인복지관 및 경로당 등의 노인복지기관과 관련한 욕구가 많은 것으로 나타남. \n- 특히, 양로시설이나 요양시설 등에 대해 높은 욕구를 보이고 있음. 이와 같은 어르신들의 복지 욕구를 해소하기 위해서는 노인전문요양시설의 건립 및 운영이 필요함. \n- 사업주관부서는 복지서비스과 노인복지팀이며, 2006년 사업예산은(국비 50%:시비50%)1,526,866,000원\n(3)사회에 기여할 수 있는 기반 조성\n□ 노인전문인력은행 및 노인 취업정보센터 등 강화\n⚬ 노인취업 지원 강화\n- 노인취업관련 사업의 지원을 강화하여 어르신들에게 맞는 일 자리를 발굴을 통해 어르신들의 일자리 창출 및 사회참여 기회 제공\n- 사업기획부서는 복지서비스과 노인복지팀이며, 노인취업관련 정보 사업지원은 대한노인회 및 노인복지센터와 연계\n□ 취업교육 확대\n⚬ 재취업을 위한 교육기회 제공\n- 현재의 노인복지회관 및 노인문화회관(중대형 경로당)에 재취업을 위한 다양한 교육 프로그램을 개설\n- 사업기획부서는 복지서비스과 노인복지팀이며, 노인복지회관 및 대한노인회 등과 연계하며 사업예산은 비예산\n4)핵심 추진사업 연차별 계획\n□ 연차별 사업계획\n⚬ 노인분야 추진사업은 노인복지기관 확충, 문화(평생)교육 강화, 노인일자리사업 확대, 재가 서비스 강화、노인의료기관 확충, 취업정보 제공 확대, 취업교육 확대로 선정하였음. \n제4절 여성복지\n1. 여성복지실태 및 현황\n1)조사 개요\n□ 조사 설계 내용\n⚬ 여성은 총 190명을 조사하였고, 대상은 계양구에 거주하는 만 20세 이상의 여성으로 선정하였음. \n⚬ 표본추출방법은 연령별, 동별, 성별에 따라 비례층화추출하였고, 조사방법은 가정방문하여 1:1개별 면접 조사하였음. \n□ 응답자 특성\n⚬ 주요 특성을 살펴보면, 직업은 전업주부가 61.2%로 조사되었고, 결혼여부에 대해서는 기혼이 85.8%로 조사되었음. \n⚬ 가구원수는 3-4명이라고 답한 비율이 74.8%로 나타났고, 주택 소유형태는 자가 주택이 82.1%로 조사되었음. \n2)여성복지실태\n□ 계양구의 여성관련 시설 이용과 관련하여 질문한 결과, 가정폭력 상담소와 여성긴급전화<|start|>#@전번1#<|end|>의 경우, 응답자 전원이 이용한 경험이 없다고 응답함. \n□ 여성관련간관 이용경험이 없는 이유는\'필요를 못느껴서\'가 응답자의 80.0%로 가장 많았으며, \'필요하지만 기관에 대한 정보를 몰라서\'(12.6%), \'기관의 접근성때문에(너무 멀거나, 차편이 부족, 기관수가 적어서)\'(5.3%), \'시간이 없어서\'(0.5%)\'의 순임. \n⚬ 이러한 결과를 통해 계양구 거주 여성들의 복지수요에 맞는 적절한 서비스를 제공 및 기관의 홍보노력이 필요하다고 볼 수 있음. \n□ 응답자들이 주어진 여성관련 문제를 본인이 얼마나 심각하게 생각하는냐의 질문에 응답한 결과를 살펴보면, 대부분 양호한 정도의 응답결과를 나타냄. \n□ 응답자들에게 여성아카데미 등 여성복지향상을 위한 프로그램에 대해 인지도 및 이용 경험을 물어본 결과, 응답자 전체의 42.1%가\'알고 있다\'고 응답하였으며, 프로그램에 참가한 경험 은 프로그램을 알고 있는 응답자의 10%가\'참가한 경험이 있다\'고 응답하였음. \n□ 위의 여성복지 향상을 위한 프로그램에 참가하지 않은 응답자의 이유를 살펴보면, \'여유시간이 없어서\'(56.8%)가 가장 많았으며, \'여가시간과 안 맞아서\'(23.0%), \'선호 프로그램이 없어서\'(13.5%), \'참가신청방법이 복잡해서\'(2.7%》, \'경제적여유가 없어서\'(I. 4%)의 순임. \n2. 복지서비스 평가 및 욕구\n1)여성복지서비스 평가\n□ 옹답자들에게 계양구의 전반적인 생활환경 만족도를 물어본 결과, 대부분\'보통\'에서\'다소 만족\'사이의 만족수준을 나타냄. \n⚬ 가장 높은 만족도를 나타낸 항목은 계양구의 교통환경(3.6)이었으며, 복지서비스가 가장 낮은 만족수준(2.9)을 나타냄. 그 외 생태환경(공기, 녹지)(3.3), 주거환경(3.3), 교육환경(3.1), 경제활동(3.0), 보건서비스(3.0)의 순임. \n□ 계양구의 여성 관련 기관들이 여성의 복지향상을 위해 어느 정도 기여하고 있다고 생각하는가의 질문에 응답한 결과를 살펴보면, 전반적으로\'보통\'의 수준보다 낮은 정도(2.8)의 응답 결과를 나타내, 다소간 기여하지 못하는 편이라고 인식하는 것으로 보임. \n□ 계양구 여성관련 시설에 대한 만족도를 조사한 결과, 가정폭력 상담소와 여성긴급전화<|start|>#@전번1#<|end|>의 경우, 응답자 모두 이용경험이 없었음. \n□ 여성 관련 기관의 이용경험이 있는 응답자들에게 기관이용에 있어 각 항목별 만족도를 알아본 결과, 대부분\'보통\'에서\'다소 만족\'사이의 응답결과를 나타냄. ⚬ 그 중, \'다른기관과의 연계\'항목에 가장 높은 만족도(4.0)를 보였으며, 그 외\'프로그램 다양성\', \'서비스의 질\', \'서비스이용료의 적절성\', \'접근성(교통)에서 큰 만족도(3.9)를 보임. \n□ 응답자들에게 계양구의 전체 사회복지서비스에 대한 만족도를 물어본 결과, 3.0으로\'보통\'의 수준으로 나타남. \n2)여성복지욕구\n□ 응답자들에게 계양구가 여성들의 복지향상을 위해 우선적으로 추진해야 할 여성복지 프로그램 및 정책을 물어본 결과, 1순위를 살펴보면, \'여성취업 활성화\'라고 응답한 결과가 가장 많았으며(30.5%), \'여성복지기관 증대\', \'보육시설 확충\', \'문화프로 그램 확대\'등의 응답결과가 많이 나타남. \n□ 또한 계양구가 여성들의 복지향상을 위해 우선적으로 추진해야 할 여성복지 프로그램 및 정책을 물어본 결과, 1순위, 2순위의 응답 결과를 합하여 살펴보더라도\'여성취업 활성화\'라고 응답한 결과가 가장 많았으며(57.9%), \'여성복지기관 증대\', \'보육시설 확충\'등 1순위 응답결과만 살펴본 바와 동일한 결과가 보임\n□ 응답자들에게 계양구가 여성들의 복지향상을 위해 우선적으로 증설해야 할 사회복지기관을 물어본 결과, 1순위로 선택한 결과를 살펴보면, \'여성복지관\'을 선택한 결과가 가장 많았으며(43.7%), \'건강가정지원센터\'(20.5%), \'가정폭력 상담실\'(16.3%), \'여성회관\'(14.2%)등의 응답결과가 많이 나타남. \n□ 또한 우선적으로 증설해야 할 사회복지기관을 물어본 결과, 1순위, 2순위의 응답 결과를 합하여 살펴보더라도\'여성복지관\'올 선택한 결과가 가장 많았으며(72.1%), \'여성회관\', \'건강가정지원 센터\'등 1순위 응답결과만 살펴본 바와 동일한 결과를 나타냄. \n□ 응답자들에게 계양구의 여성복지 수준 향상을 위해 계양구에 건의하고자 하는 의견이 있는지 물어본 결과, 아래 표와 같은 의견들이 제시되었음. \n3)보건의료 관련 실태 및 평가\n□ 응답자들의 한 달 평균 의료비를 조사한 결과, 계양구 여성들의 월 평균 의료비는 응답자 전체 평균 2만9천8백원이며, 응답 분포는 1만원 이하(42.1%)가 가장 많았으며, 1~2만원(26.3%), 2~3만원(13.2%), 3~5만원(10.0%), 5만원 이상(8.4%)의 순임. \n□ 응답자에게 아플 때 주로 어떻게 대처하는 지를 물어본 결과, 대부분 병•의원을 이용한다는 응답(74.2%)을 하였음. 다음으로 약국을 이용하는 경우(19.5%), 그냥 참음(3.2%), 대형병원을 이용하는 경우(2.1%), 보건소를 이용하는 경우(1.1%)의 순임. \n□ 응답자들은 아플 때 치료 받기 원하는 기관으로 개인 병•의원이라고 응답한 결과(67.9%)가 가장 많았으며, 종합병원(23.2%), 약국(5.8%), 보건소(2.6%), 기타(0.5%)의 순임. \n□ 치료받고 싶은 의료기관과 현재 주로 이용하는 치료방법과 차이가 있다면 무슨 이유 때문인지 물어본 결과, 차이가 있는 경우(32.1%), \'병이 크지 않아서\'(28.9%), \'치료비가 부담되서\'(17.4%), \'대기시간이 길어서"(9.3%), 거리가 멀어서(4.7%)의 순임. \n3. 여성복지계획\n1)계양구 여성복지 사업현황 및 평가\n□ 2006년 계양구 여성복지 사업 현황\n□ 여성복지부문 평가\n⚬ 계양구 여성복지부문 사업현황을 살펴보면, 모•부자가정 지원에 많은 부분 지원되고 있고 일반 여성에 대한 복지사업은 거의 없어 그들의 복지욕구를 충족시키지 못함. \n⚬ 대부분의 복지예산이 취약계층 복지부분에 소요되고 있어 여성복지부문의 예산은 일반주민의 삶의 질 제고에는 그다지 기여하지 못하는 것으로 평가됨. \n2)여성복지 정책목표\n□ 여성복지부문의 정책목표는\'건강한 여성•행복한 가정 만들기\'로 설정하였음. \n⚬ 욕구조사결과 여성복지회관에 대한 수요가 높았고 가족프로그램, 문화프로그램, 보육관련 시설에 대한 욕구들이 많은 것으로 확인되고 있으나 현재 여성복지서비스에 대한 평가는 매우 낮은 것으로 나타남. \n⚬ 이에 본 계획에서는 취약계층에 대한 복지사업은 지속적으로 강화하면서 일반주민의 복지욕구를 충족시킬 수 있는 정책목표 및 정책부문, 부문별 추진목표를 설정함. \n3)여성복지 중점추진사항\n(1)여성의 삶의 질 제고\n□ 문화• 평생교육 강화\n⚬ 종합복지회관에 여성들이 참여할 수 있는 문화•교양 •기술 등 평생교육활동 프로그램 개설. \n- 교양프로그램은 기초교양위주의 프로그램을 개설하여 지역 내 여성인구의 복지 욕구 충족. \n- 지역 내 대학과 연계하여 지역 내 여성주민이 이용할 수 있는 강좌 개설(기존 평생교육시설에 프로그램 확대 및 지원강화). \n- 사업기획부서는 복지서비스과 여성•아동복지팀이며, 프로그램 개설기관과 연계. \n⚬ 여성아카데미 확대 강화\n- 기초교양강좌 보다는 심화 교양과정을 개설하여 지역주민의 평생교육에 대한 욕구를 충족시키는 방향으로 설정. \n- 일회성의 강연보다는 학습프로그램이 될 수 있도록 지속적인 장기프로그램으로 개설하여 실제적인 지적능력 증대와 실생 활에 이용가능하도록 설계. \n- 사업주관부서는 복지서비스과 여성•아동복지팀. \n□ 성폭력 예방 활동 강화\n⚬ 성폭력 예방 교육 지원 및 강화\n- 성가치 교육과 폭력사태 시 현실적 대처방안교육을 위한 출장 교육전문가 육성지원. \n- 주요 사업내용은 각 기관별 성교육의 체제 강화하고, 가정 내에서 바른 성폭력 예방교육을 부모님이 실시 할 수 있도록 부 모님들께도 교육기회 제공. \n- 사업주관부서는 복지서비스과 여성•아동복지팀이며 사업예산 은 성희롱 교육과 관계된 예산을 활용하고, 연차적으로 확대. \n⚬ 가정 내 성폭력 예방\n- 부부 상담 기관 연계성 프로그램 운영. \n- 사업주관부서는 복지서비스과 여성•아동복지팀이며 사업예산 은 가정폭력상담소예산에서 충당하고 확충되는 인력은 상담 역할과 함께, 유관기관과의 연계역할을 수행함\n(2)여성의 사회활동 기회 확대\n□ 취업 정보제공 및 기회개발\n⚬ 여성의 자기개발 지원\n- 종합사회복관 교육프로그램 개발에 있어 저소득 여성이 쉽게 배워 취업할 수 있는 도배, 풍선인테리어, 데코레이터 등의 교육프로그램이 포함될 수 있도록 개발. \n- 사업기획부서는 복지서비스과 여성•아동팀이며 종합사회복지관 둥에 위탁하여 시행. \n⚬ 취업 정보 제공\n- 계양구 홈페이지를 이용하여 여성들이 취업 정보를 획득할 수 있도록 하여 지역 내 여성들에게 취업정보 제공. \n- 사업주관부서는 복지서비스과 여성•아동팀. \n(3)행복가정지원육성\n□ 행복가정지원 활동\n⚬ 행복가정 상담 도우미 강화\n- 행복가정상담 도우미 활동을 강화하여 가족문제와 관련된 상담 및 컨설팅 활동. \n- 기존의 행복한 가정만들기 상담도우미 제도를 확대운영하는 것으로서 분야별 전문 상담인력을 확충하여 시행. \n- 사업기획부서는 복지서비스과 여성•아동팀이며, 사업집행은 위탁함\n• 교육프로그램 중 행복한 가정을 위한 강좌개설\n- 올바른 자녀 양육을 유도하는 부모 교육 및, 가족해체 예방을 위한 부부대상교육 등을 개설\n- 사업주관부서는 복지서비스과 여성•아동팀이며, 종합사회복지관 등에 위탁하여 사업시행. \n- 사업예산은 각 복지관 운영비를 통하여 충당. \n□ 모•부자 가정 지원 강화\n⚬ 모•부자가정 자녀 문화•과학체험활동 우선권 보장\n- 구청 주관의 문화행사 및 각종 체험활동 등에 모부자가정에 정보를 제공하여 선택에 있어 우선권을 보장. \n- 사업기획부서는 복지서비스과 여성•아동팀이며 연계부서 및 기관은 문화공보과 및 복지기관 등\n⚬ 한부모가정 개선 프로그램 신설\n- 지역내 독거 노인•노인 단독세대, 농촌지역 거주자 및 기타 희망자(가족)등과 모•부자가정간의 결연사업 실시. \n- 사업기획부서는 주민생활지원과 서비스연계팀이며, 연계부서 및 기관은 자원봉사센터, 노인복지회관 등. \n□ 가족 프로그램 확대 강화\n⚬ 가족과 함께하는 생태•문화 체험 사업\n- 청소년회관, 종합사회복지관 등 각종 사회복지기관에서 기관의 성격에 따라 프로그램을 제공할 때, 가족단위 참여가 가능 하도록 설계. \n⚬ 가족자원봉사 프로그램 확대\n- 가족의 해체와 지역사회문제를 예방하기 위하여 가족공동의 봉사활동으로 유도할 수 있는 프로그램을 개발하여 가족단위의 봉사활동 활성화 유도. \n- 사업기획부서는 자원봉사센터에 위탁하여 시행. \n- 사업예산은 자원봉사센터 확대 예산을 통해 충당. \n4)핵심 추진사업 연차별 계획\n□ 연차별 사업계획\n⚬ 여성복지 분야 추진사업은 여성사회복지 확대, 여성평생교육, 여성취업정보제공 및 개발 확대, 가족단위 체험 프로그램 확대로 선정하였음. \n제5절 장에인복지\n1. 장애인복지실태 및 현황\n1)조사개요\n□ 조사 설계 내용\n⚬ 장애인은 총 150명을 조사하였고, 대상은 장애인 명부를 기준으로 하여 대상자를 선정하였음. \n⚬ 표본추출방법은 연령별, 동별, 성별에 따라 비례층화추출하고, 이를 명부에 기초하여 계통 추출하였음. \n⚬ 조사방법은 조사가능이라고 대답한 대상자에 대해서 1:1 가구 방문 면접조사를 실시하였고, 일부는 전화 조사하였음\n□ 응답자 특성\n⚬ 주요 특성을 살펴보면, 연령은 50대 이상이 33.3%로 가장 많은 것으로 나타났고, 직업여부 중 무직이 43.0%로 조사되었음. \n⚬ 가족 중 장애 및 질병을 가진 사람이 있느냐라는 질문에는 38.0%가 "있다"라고 대답하였고, 가구소득은 200만원이하라고 답한 비율이 68.7%로 조사되었음. \n2)장애인복지실태\n□ 본인의 장애 수용정도를 조사한 결과\'그저 그렇다\'가 47.3%, \'수용하는 편이다\'와 매우 잘 수용하고 있다 38%로, 응답자 대부분이 자신의 장애를 수용하고 있는 것으로 나타남\n□ 본인의 장애 정도의 악화 가능성에 대해 조사한 결과 현 상태를 유지할 것이라는 응답이 53.3%, 지금보다 더 나빠질 것 34.7%, 아주 나빠질 것 2.0%로 응답자의 90.2%가 현 상태를 유지하거나 더 나빠질 것이라고 응답함. \n□ 본인의 장애를 치료하고 있는지를 조사한 결과, 치료하고 있는 경우는 22.2%, 치료하고 있지 않는 경우는는 77.8%로 나타남. \n□ 위의 조사에서 치료를 받지 않는다는 응답자의 이유를 살펴보면, 의사의 지시라고 응답한 52.9%를 제외한 나머지의 경우 치료비가 부담되어서 23.0% 스스로 치료가 불필요하다고 판단 하여서 19.5% 적절한 치료기관이 없어서 2.3%로 나타남. \n⚬ 따라서, 응답자의 44.8%가 치료가 필요함에도 적절한 치료를 받고 있지 않을 가능성이 있는 것으로 판단됨. \n□ 장애인이 현재 치료받는 기관을 살펴보면, 일반병원이 78.1% 한방병의원이 6.3%로 응답자의 84.4% 사적영역에서 치료를 받고 있는 것으로 나타남. \n⚬ 치료받고 싶은 기관 역시 일반병원과 한방병원은 64.1%, 재활 전문기관이 28.1%로 나타나 재활전문기관의 확대가 필요함\n⚬ 다만 장애인 복지기관에서 치료를 받고는 경우는 4.7%인데 비해 장애인 복지기관에서 치료를 받고자 하는 경우는 1.6% 로 나타나 이에 대한 검토가 필요함. \n□ 한 달에 재활치료로 소요되는 비용은, 응답자의 54.7%가 5만원 미만이라고 응답하였고, 10만원 이상 이라고 응답한 경우는 21.9%를 차지함. \n• 재활치료에 소요되는 금액 평균은 10.17만원으로 나타났고, 작전서운동이 24만 1천 7백원으로 가장 많은 금액을 사용하고 있으며, 효성동이 4만원으로 가장 낮은 비용을 지출함. \n□ 장애인에게 일상생활에 도움이 필요한지를 조사한 결과 59.3%는 혼자 생활이 가능하다는 응답자가 59.3%이고 많은 도움이 필요하다는 응답과 전적인 도움이 필요하다는 응답이 18.7%로 나타남. \n□ 장애인의 일상생활 보조자를 살펴보면, 배우자(47,5%), 직계가족(14.8%), 자녀의 배우자(13.1%), 형제•자매(11.5%)로 가족에게 의지하는 경우가 86.9%로 장애인 대부분이 가족에게 도움을 받고 있는 것으로 나타남. \n• 자원봉사자의 도움을 받는 경우는 4.9%에 불과해 일상생활 부분에 있어 복지서비스 전달에 대한 재평가 요구됨. \n□ 가정봉사 서비스 이용의사가 있는지 조사한 결과 응답자의 68.8%가 이용의사가 없는 것으로 나타남. \n□ 장애인의 사회활동 참여 정도를 살펴보면, 가끔 참여 한다(49.3%), 자주 참여한다(15.3%), 매우 자주 참여한다(9.3%)로 나타났으나, 사회활동을 전혀 하지 않는다는 응답자도 24.7% 를 차지해 이들에 대한 적절한 대응이 요구됨. \n□ 직업 활동을 제외한 사회활동에 참여하는 장애인의 활동 내용을 살펴보면, 전체 응답자의 73.0%가 친구•이웃과 만남이라고 응답하였고, 25.2%는 친척과의 만남, 봉사활동 1.8%로 나타남. \n⚬ 스포츠•레저활동, 쇼핑•여행 등의 활동은 전무한 것으로 나타나, 장애인의 적극적인 사회활동 참여 유도를 위한 대응이 요구됨\n□ 장애인의 여가활용 방법을 살펴보면, TV시청이 84.0%로 가장 높은 비율을 차지했고, 인터넷•컴퓨터가 6.7%, 사회활동이 5.3%, 여가활용 없음이 2.0%로 나타남. \n⚬ 여가활동으로 TV시청과 인터넷•컴퓨터를 이용한다는 응답자가 90.7%로 차지해 대부분의 장애인은 집안에서 여가활동을 하는 것으로 나타남. \n⚬ 특히, 작전서운동과 계양1동은 응답자 전원이 여가활용으로 TV시청을 한다고 응답하여 작전서운동과 계양1동 거주의 장애인에게 여가활용을 위한 프로그램 제공이 요구됨. \n□ 장애인이 선호하는 교통수단을 살펴보면 전철 32.7%, 자가용 27.3%, 콜택시•전용수단 20.7%, 버스 16.0%로 나타나 응답자의 48.7%가 대중교통을 선호하는 것으로 나타남. \n□ 계양구의 장애인이 복지서비스에 대한 정보를 획득하는 경로를 살펴보면 신문•TV•라디오 38.7%, 동사무소 22.7%, 가족•친척 22.0%로 나타나 대중매체가 주된 정보획득 수단임. \n⚬ 계양2동과 작전서운동은 동사무소를 통한 복지정보 획득 경로가 각각 50.0%와 38.5%를 차지하여 대중매체에 의한 정보 획득 비율을 초과함, 이들 동사무소의 복지정보 전달과 관련 된 정보공유 등을 검토할 필요가 있음. \n□ 장애인이 일상생활에서 겪고 있는 어려움을 살펴보면, 건강문제, 경제문제, 직업•경제활동, 의료서비스이용에서 어려움을 겪고 있는 것으로 나타남. \n⚬ 반면, 가족관계, 성욕구 해소, 이성교제•결혼, 음주문제 등 개인적인 부분에 대한 어려움은 크지 않은 것으로 나타났으나 이는 개별적인 접촉과 전문 인력의 상담 등을 통해서만 확인이 가능할 것으로 판단됨. \n□ 폭력, 유기•방치, 학대 등 장애인이 겪고 있는 문제를 살펴보면, 설문조사 결과는 가장 낮은 점수의 언어폭력 4.7에서 가장 높은 점수의 성적학대 5,0으로 동간의 차이 없이 모두\'거의 문제가 없는 것\'으로 나타남. \n□ 보건복지 관련 시설에 대한 인지여부를 살펴보면, 각 시설 중\'알고 있다\'는 응답은 일반 병•의원이 94.7%, 보건소 94.7%로 가장 잘 인지하고 있는 것으로 나타남. \n□ 각 보건복지 관련 시설에 대해\'모른다\'는 응답은 공동생활가정 96.0%, 수화통역센터 88.0%, 심부름센터 87.3%, 주간단기보호시설 84.0%, 정신장애인복귀시설 82.0%, 특수학급 81,3%, 전 용체육시설 81.3%, 정신보건센터, 77.3%, 특수교육기관 68.0, 생활요양시설 647%, 직업재활시설 62.7%, 직업훈련원 58.7%, 특수학교 56.7% 순으로 나타남. \n□ 보건복지 관련 시설 중 가장 높은 인지율을 보인 일반 병•의원과 보건소의 이용 경험이 없다는 응답도 각 83.3%에 불과함. \n⚬ 보건복지 관련 시설 중 공동생활가정, 주간단기보호시설, 심부름센터, 수화통역 센터, 정신장애인복귀시 설, 정신보건센터, 특수학교, 특수학급 등은 이용경험자가 0%로 나타남. \n□ 보건복지 관련 시설 필요도를 살펴보면, 일반 병•의원(4.1)재활 병•의원과 보건소(4.0)로 가장 높게 나타났고, 수화통역센터(2.6), 정신장애인복귀시설과 특수학급(2.8), 정신보건센터•특수학교•특수교육기관(2.9)로 상대적으로 필요도가 낮은 것으로 나타남. \n□ 보건복지관련 시설의 이용 만족도를 살펴보면, 보건복지관련 시설 종사자의 친절도가 3.4로 가장 높고 상대적으로 이용 요금의 적절성, 물리적 환경 및 편의시설, 이용거리 및 교통 등 의 평가는 3.1 로 낮게 나타남. \n□ 보건복지 시설을 이용하지 않는 이유를 살펴보면, \'이용할 필요가 없다\'는 응답이 37.3%로 가장 큰 빈도를 보였고, \'기관에 대해 잘 몰라서\'라는 응답이 30.0%로 그 다음 빈도를 보였음. \n□ 보건복지 프로그램 및 서비스 인지도를 살펴보면, 영구임대주택 우선권 등 주거서비스에 대해\'알고 있다\'는 응답이 83.3%로 가장 높은 인지도를 보였고, 장애관련 수당, 세금공제 등 소득보장이 77.1%, 심리상담 및 훈련 등 심리재활 서비스 55.6% 등 비교적 높은 인지율을 보임. \n⚬ 반면, 외출도우미•교통비할인 등 교통•이동서비스는\'모른다\'고 응답한 경우가 61.1%, 가정봉사원 파견과 밑반찬전달 등 재가서비스는 55.6%가\'모른다\'고 응답하여 비교적 낮은 인지율을 보임. \n□ 보건복지 프로그램 이용한 경험에서는 의료재활이 46.7%, 정보 제공 11.1%의 빈도로 나타남. \n⚬ 반면 심리재활, 여가프로그램 등은 이용 경험자가 없는 것으로 나타나 이들 서비스에 대한 보다 적극적인 홍보를 통해 이용을 유도해야 함. \n□ 보건복지 프로그램을 이용하지 않는 원인을 살펴보면, 필요하지만 해당 서비스에 대해서 몰라서가 26.0%로 가장 높은 빈도를 보였고, 다음으로 필요가 없어서와 이용방법을 몰라서가 25.3%의 빈도를 보임. \n⚬ 보건복지 프로그램도 보건복지 관련 시설과 마찬가지로 프로그램에 대한 홍보 강화로 이용을 유도해야 함. \n2. 복지서비스 평가 및 욕구\n1)장애인복지서비스 평가\n□ 계양구의 일반적 생활환경에 대한 장애인의 평가를 살펴보면, 전체 응답자의 86.6%가\'보통 이상\'이라고 응답함. \n⚬ 작전서운동과 계양1•2동은 각 15.4%, 16.7%, 22.2%의 불만족 빈도를 보여 이들 지역의 일반 생활환경의 점검이 요구됨\n□ 장애인과 관련된 생활환경 평가는 만족한다와 매우 만족한다가 18.07%, 불만족하다와 매우 불만족하다가 38,7%로 나타나 불만족하다는 응답이 더 높은 빈도를 보임. \n⚬ 특히, 작전서운동과 계양2동의 불만족하다는 응답이 각 61.5%, 55.6% 로 나타나 타 동의 장애인에 비해 장애인 관련 생활환경에 많은 불편을 느끼는 것으로 판단됨. \n□ 장애인 관련 생활환경 불만족 요인을 살펴보면, 복지시설이 부족하다는 응답이 55.2% 로 가장 높은 빈도를 보였으며, 재활시설부족이 25.9%, 이동편의시설 부족이 13.8%, 교육기관 부족이 5.2%로 나타남. \n⚬ 효성동과 계양2동은 복지시설이 부족하다는 응답과 이동편의 시설이 부족하다는 응답의 빈도가 높게 나타나 이들 지역의 복지시설과 이동편의 시설의 확대가 요구됨. \n⚬ 교육기관이 부족하다는 응답은 작전동 22.2%와 계양동 10.0%의 빈도만은 보여 이들 지역의 교육기관의 확충이 필요하다고 판단됨. \n□ 장애인의 계양구 전체 사회복지 서비스 만족도 조사 결과를 살펴보면, \'만족하다\'10%, \'불만족하다\'와\'매우 불만족하다\'가 25.3%로 나타나 계양구 사회 복지 서비스에 만족한다는 응답보다 불만족하다는 응답이 더 높은 빈도를 보임. \n⚬ 다만, 보통이라는 응답이 64.7%를 차지하여 전체 평균점수는 2.8로 나타나 계양구 사회복지 서비스에 큰 불만은 없는 것으로 나타나며, 동별 만족도의 차이는 크지 않음. \n□ 장애인이 체감하는\'복지기관의 장애인에 대한 관심정도\'는 평균 2.8로 나타나 보통에 근접한 결과를 보이며, 다만 작전서운동이 3.4점으로 가장 높게 평가하였고, 효성동이 2.4점으로 가장 낮게 평가함. \n□ 장애인의 거주 주택에 대한 편리성 평가를 살펴보면, 매우 편리하다는 응답과 대체로 편리하다는 응답이 63.3%였으나, 약간 불편하다는 평가와 매우 불편하다는 평가가 36.7%를 차지해 장애인 거주 주택 개•보수 등의 지원이 필요하다고 판단됨. \n⚬ 특히, 약간 불편과 매우 불편하다는 응답은 계양2동(44.5%), 계양1동(41.7%), 효성동이(400%)의 순으로 높은 빈도를 보여 이들 지역의 주택 및 주거환경에 대한 점검이 필요함. \n2)장애인복지욕구\n□ 의료서비스, 경제적 어려움, 차별적 인식•태도, 직업•사회참여, 문화생활영위, 폭력예방 중 선결되어야 할 문제에 대한 장애인의 평가를 1순위 응답 수준에서 살펴보면, 차별적 인식•태도가 38.0%로 가장 높은 빈도를 보였고, 그 다음으로 의료서비스가 31.3%의 빈도를 차지함. \n⚬ 의료서비스에 대해서는 효성동(46.7%), 계양2동(44. 4%), 작전동(34.6%)의 순으로 선결되어야 할 복지로 나타남. \n□ 장애인이 선택한 선결 복지서비스 1,2,3순위를 합산한 결과를 살펴보면 경제적 어려움 81.3%, 의료 서비스 72.0%, 직업•사회참여 44.7%, 차별적 인식•태도 42.7%, 이동•교통문제 33.3% 순서로 나타나, 경제적 어려움이 가장 많은 응답자가 선결되어야 한다고 응답하였음. \n□ 장애인 복지 증진을 위해 역점을 두어야 할 사업을 1순위로 살펴보면, 32.0%의 빈도를 보인 의료재활 그 다음으로 16.0%의 빈도를 보인 주거서비스, 15.3%인 직업재활 순으로 나타남. \n□ 장애인 복지 역점 사업을 1,2,3순위를 합산하여 살펴보면 의료재활 70%, 소득보장 50%, 주거서비스 49.3%, 직업재활이 36%의 빈도를 보여, 장애인 복지를 위해 역점을 두어야 할 사업으로 의료재활이라는 응답의 빈도가 가장 많음을 알 수 있음. \n□ 장애인 교통편의를 위해 지원되어야 할 수단을 조사한 결과, 대중교통이용지원이 36.7%, 장애인 전용 교통수단 제공이 34.0%, 장애인 교통수당 제공이 28.0%로 나타나 다수의 장애인이 대중교통이용지원이 필요하다고 응답함. \n⚬ 다만 작전서운동, 계양1동, 계양2동은 장애인 전용 교통수단 제공이 가장 높은 빈도를 보였고, 작전동에서는 장애인 교통수당제공이 가장 높은 빈도를 차지함. \n3. 장애인 복지계획\n1)계양구 장애인 사업현황 및 평가\n□ 2006년 계양구 장애인 복지 사업 현황\n□ 장애인 복지부문 평가\n⚬ 계양구 장애인 복지부문은 장애인 시설부분에 많은 예산이 투입되고 있는 것으로 나타남. \n⚬ 그러나 장애인에 대한 인식전환사업 부분에는 별다른 프로그램이 마련되어 있지 않고, 예산투입도 적어 이 부분에 대한 예산지원이 필요한 것으로 평가됨. \n⚬장애인 복지욕구 조사 결과를 살펴보면 일반생활환경에 대한 만족도는 보통 60%, 만족 26.6%로 응답해 만족스럽다는 응답이 불만족스럽다는 응답보다 높지만, 장애인 생활환경의 경우 보통이 42.7% 불만족 38.7%로 응답하여 불만족스러운 것으로 평가함. \n⚬ 장애인 관련 생활환경 불만족 요인으로는 복지시설의 부족, 재활시설 부족을 꼽고 있어 이에 대한 복지수요가 많은 것으로 판단됨. \n2)장애인 복지 정책목표\n□ 장애인 복지 정책목표는 "장애인 복지 네트워크 구축을 통한 보건의료서비스 강화"와 "사회의 주체적 일원으로서 사회 참여기회 확대"로 설정함. \n⚬ 욕구조사결과 의료재활, 소득보장, 주거서비스 등이 장애인 복지 역점사업의 주요 욕구로 파악되었고, 일상생활과 관련된 문제에 대해서는, 경제적 어려움, 의료문제, 직업•사회참여, 이동•교통문제 등이 심각한 것으로 나타남. \n⚬ 이에 본 계획에서는 주요정책부분을 장애인 지원체계를 구축, 사회참여기회를 확대, 사회복지서비스와 보건의료서비스 연계 강화 등으로 설정함. \n3)장애인 복지 중점추진사항\n(1)장애인 지원체계 구축\n□ 인식전환 사업 확대\n⚬ 좋은세상만들기 베스트업소 및 친절인 표창 사업 확대\n- 현재 시행중인 좋은세상만들기 베스트친절인(업소)는 년4회 10명씩 표창장 수여에 그치고 있어, 인식전환을 위한 사업의 실효성이 욕구조사결과에 비추어보더라도 의심됨. \n- 이를 보완하기 위해 사업 횟수는 년2회로 하되, 실효성 있는 상금 또는 상품을 제공. \n- 베스트업소(장애인 이용편리 업소)의 경우 복지소식지를 통해 장애인들이 이용할 수 있도록 홍보. \n□ 장애인 재활 및 자립 지원\n⚬장애인 거주환경 개선사업\n- 혼자서는 일상생활이 불편한 중증 장애인이 약 20% 정도인 것으로 파악되고 있으며 이들에 대한 실질적인 지원으로서 주거환경개선사업이 필요한 것으로 판단됨. \n- 현재 14개의 사업계획이 있으나, 욕구조사에서 주거환경에 대해 36.7%가 불편하다고 답해 주거환경 개선사업의 지속적인 확대•유지가 필요함\n- 사업주관부서는 복지서비스과 장애인복지팀이며, 예산은 복권기금과 시비에서 충당. \n⚬ 지체장애인 생활시설 건립\n- 계양구에는 2005년 보건복지부 통계상 장애인 생활•요양시설이 전무한 것으로 나타나고 있고 욕구조사 결과 이에 대안복지욕구가 높은 편으로 조사되고 있어 장애인 생활시설 건립이 필요한 것으로 판단. \n- 장애인 생활시설 건립의 주관부서는 복지서비스과 장애인복지팀이며, 시설건립 후 위탁 운영. \n- 사업예산은 국비 및 시비를 통해 확보. \n□ 저소득 장애인 지원 확대\n⚬ 자립자금대여 대상자 확대\n- 자립자금대여 제도 정보 제공과 수요자 발굴을 통해 자립자금대여 대상자를 확대. \n- 사업주관부서는 복지서비스과 장애인복지팀\n⚬ 주거환경 개선사업 우선 적용\n- 주거환경 개선사업 신청자 중 저소득 장애인에 대하여 주거환경 개선사업을 우선 적용함. \n- 사업주관부서는 복지서비스과 장애인복지팀. \n(2)사회참여 기회확대\n□ 직업교육 및 취업지원체계구축\n⚬ 직업재활시설 확대\n- 장애인의 취업훈련을 위한 직업훈련시설을 1개소 신설 후 취업훈련 프로그램을 다양화하여 내실 있게 운영. \n- 직업훈련시설을 통해서 장애인에게 필요한 직업훈련을 제공하여 일반사업장, 근로작업시설, 보호작업시설 등으로 취업을 알선하고 일반사업장에 취업이 힘든 장애인에게는 보호작업시설을 통해 취업을 할 수 있도록 함. \n- 사업주관부서는 복지서비스과 장애인복지팀, 장소는 임대를 통해 확보하고 사업예산은 시비 및 분권교부금을 통해 충당하며, 기존의 장애인 재활시설과 비슷한 규모로 운영하고 운영방식은 민간위탁방식. \n⚬ 장애인취업정보시스템\n- 현재 노틀담복지관의 장애인 취업관련 기능을 확대하고 직업조사 및 기관연계를 위한 전문인력을 연차적으로 확보할 수 있도록 하여 장애인에게 내실 있는 취업정보를 제공. \n- 사업주관부서는 복지서비스과 장애인복지팀이며 연계기관은 노틀담복지관. \n□ 여가지원서비스제공\n⚬ 나들이 행사의 내실화\n- 복지욕구 조사를 살펴보면 사회활동참여 여부에 대한 질문에 참여 없음이나 가끔 참여한다는 응답이 74%로 나타나고 있고, 일상생활에 전적인 도움이 필요로 하는 경우가 6.7%로 조사되었음. \n- 그러므로 사회활동참여에 대한 욕구가 높은 것으로 판단이 되며 이에 중증장애인 뿐만 아니라 경증 장애인을 포함하는 나들이사업을 시행. \n- 계양구청 주관의 나들이 행사에 있어 서비스 중복을 피하기 위해 연간 중복이용자가 발생하지 않도록 보건소, 노틀담복지관 등과 연합행사 및 이용자 정보교류 등의 연계 확대\n⚬ 저상승합이동차량 확충\n- 욕구조사결과를 살펴보면 6.7%가 일상생활에 전적인 도움이 필요한 것으로 조사되었고, 많은 도움이 있어야만 일상생활이 가능하다는 응답층도 12%로 조사되어 이동권확보가 필요한 것으로 보임. \n- 교통이동서비스를 이용해보았다는 응답자는 7.7%에 불과해 이에 대한 수요가 있음. \n- 사업주관부서는 복지서비스과 장애인복지팀, 장애인용 승합차는 2008년에 1대를 확충하고 사업예산은 임시일용인부임(운전기사)및 차량유지비 등을 포함하여 5,000만 원. \n□ 장애인 자원봉사 지원체계 구축\n⚬ 장애인 자원봉사대 활성화\n- 중증장애인을 위한 여가서비스, 재가서비스 등 다양한 부문에 봉사인력이 필요함. \n- 현재, 민간자원봉사대가 있으므로 구청 및 자원봉사센터와의 연계를 통해 장애인 자원봉사의 내실화. \n- 사업주관부서는 복지서비스과 장애인복지팀이며 연계기 관은 자원봉사 센터. \n⚬ 자원봉사자에게 인센티브 제공\n- 장애인 자원봉사대원에게는 차량 등에 표시를 부착하여 계양구 시설이용 할인제도 등을 도입하여 장애인 자원봉사를 적극 유도. \n- 사업주관부서는 복지서비스과 장애인복지팀이며 연계부서 및 기관은 시설관리공단 등. \n(3)사회복지와 보건의료서비스와의 연계 강화\n□ 재활 프로그램 및 시설 강화\n⚬ 의료재활전문기관 유치\n- 욕구조사 결과 장애인이 이용하는 치료기관은 일반병원이 78.1% 이고 재활전문기관은 6.3%인 반면 치료받기 원하는 기관은 재활 전문기관이 28.1%로서 재활전문기관에 대한 수요가 많음. \n- 따라서 의료재활전문기관이 관내에 설립될 수 있도록 다양한 인센티브를 제공. \n- 사업기획부서는 복지서비스과 장애인복지팀이며 보건소 및 자치행정과 사회협력팀 및 홍보팀 등과 연계. \n⚬ 직업재활 강화\n- 앞서 제시한 직업훈련시설 1개소에 직업재활프로그램을 다양화하여 장애인 재활지원 강화. \n□ 재가 복지 및 의료서비스 확대\n⚬ 재가 서비스 담당 사회복지사 및 자원봉사자 확충\n- 현행 35명(사회복지전담공무원 20명)의 공무원 및 자원봉사자가 재가서비스를 담당하고 있는데, 복지수요를 살펴보면 응답자의 31.2%가 장애인의 가정봉사서비스 이용의향을 가지고 있음. \n- 현재 계양구 등록장애인은 9,982명으로서 이중 31.2%인 약 3,000여명이 가정봉사서비스 수요자로 파악됨. 그러나 현재 재가서비스 담당인원은 매우 부족한 실정임. 이에 재가서비스 담당 사회복지사 및 자원봉사자의 확충이 필요함. \n- 사업기획부서는 복지서비스과 장애인복지팀이며 연계기관은 재가복지센터이며, 시비와 분권화교부금을 통해 충당하고 부족분은 구비로 충당. \n⚬ 보건소 방문보건팀 확충\n- 현재 보건소의 방문보건팀은 총10명으로 구성되어 있으나, 수요를 고려할 경우 인력의 부족으로 충분한 서비스 제공이 되고 있다고 판단하기 어려움. \n- 따라서 보건소 및 지역보건의료계획에 이를 반영하여 재가의료서비스의 양•질적 수준 제고. \n4)핵심 추진사업 연차별 계획\n□ 연차별 사업계획\n⚬ 장애인 분야 추진사업은 인식전환사업 확대, 장애인 자립 지원, 직업교육 및 취업지원체계구축, 여가지원 서비스 제공, 장애인 자원봉사지원체계구축, 재활 프로그램지원체계구축, 재활 프로그램 및 시설강화, 재가 복지 및 의료서비스 확대로 선정. \n제6절 저소득층 복지\n1. 저소득층 복지실태 및 현황\n1)조사개요\n□ 조사 설계 내용\n⚬ 저소득층은 총 180명을 조사하였고, 대상은 저소득층 명부를 기준으로 하여 대상자를 선정하였음. \n⚬ 표본추출방법은 연령별 동별 성별에 따라 비례층화추출하고, 이를 명부에 기초하여 계통 추출하였음. \n⚬ 조사방법은 조사가능이라고 대답한 대상자에 대해서 1:1 가구 방문 면접조사를 실시하였고, 일부는 전화 조사하였음. \n□ 응답자 특성\n⚬ 주요 특성을 살펴보면, 연령은 50대 이상이 39.4%로 가장 많은 것으로 나타났고, 직업여부 중 무직이 32.8%로 조사되었음. \n⚬ 장애 및 질병여부에는 만성질환자가 15.0%로 조사되었고, 주택 소유형태는 전세가 41.7%로 가장 많이 차지하였음. \n2)저소득층 복지실태\n□ 저소득층의 복지수혜 경험을 살펴본 결과 가장 많이 제공받고 있는 복지서비스는 생계급여 지원, 의료급여지원, 자녀교육비지원 순으로 나타났음. \n□ 저소득층의 가족들의 경우 음주문제(4.8)가 가장 심각한 문제로 나타났으며 다음으로 가족 간 불화(4.4), 자녀교육문제(3.7)등의 문제 순으로 나타남. \n□ 저소득층이 사용하는 월 평균 의료비는\'1만원 이하\'가 전체 응답자의 53.3%가 응답을 하였으며 평균 의료비는 3만3천6백원으로 나타났음. \n2. 복지서비스 평가 및 욕구\n1)저소득층 복지서비스 평가\n□ 저소득층의 경우 생활환경에 대해서 대부분 평균을 약간 상회 하는 만족을 표시하는 것으로 나타났고, 이 중 경제환경(2.8)의 경우 다소 불만족하고 있는 것으로 나타났음. \n□ 계양구 사회분야 만족도에 대해서 환경분야(3.2)를 제외하고 전 항목에 대해서 불만족을 표시했으며 그 중 소득격차와 사교육비 문제에 대해서 불만족한 평가를 하는 것으로 나타났음. \n□ 전체 사회복지서비스에 대한 평가에서 응답자 180명이\'보통\'정도로 평가하고 있으며, 동별로 파악했을 경우 효성동(2.8), 작전동(2.9), 계양2동(2.8)로 다소 불만족스러운 것으로 평가하였음. \n□ 응답자가 가장 많이 제공받은 서비스로는 생계급여(166명), 의료급여지원(151명), 자녀교육비지원(56명), 결식학생급식지원(49 명)순으로 나타났으며 생계급여(2.8)는 다소 불만족, 의료급여 지원(3.1)로 보통, 자녀교육비지원(3.1)로 보통, 결식학생급식지 원(3.3)으로 보통을 약간 넘는 것으로 나타났음. \n□ 저소득층이 받는 서비스에 대한 만족도의 경우, 52.2%가 보통으로 응답하였으며\'다소 만족\'의 경우 25%가\'다소 불만족\'의 경우 응답자의 21.7%로 나타났음\n2)저소득층 복지욕구\n□ 향후 4년간 계양구 저소득층 선결 사업 1순위는 생계비지원이 1순위(59.4%)로 꼽았으며 2순위는 임대주택 등과 같은 주거환경지원(11.7%)로 나타나 저소득층에 대한 생계비지원과 주거환경개선에 역점을 두어야 할 필요성이 제기됨. \n□ 저소득층에게 1순위부터 3순위까지 선택하게 한 경우 현금 등의 생계비지원이 1순위(90.6%), 2순위(50.6%)인 자녀교육비•양육지원, 3순위(48.3)는 의료비 지원으로 나타나 계양구 저소득층 복지계획수립 시 이를 염두해야 할 필요성이 있음. \n3)보건의료관련 실태 및 평가\n□ 저소득층의 경우 몸이 불편한 경우 대부분이 병 의원(73.3%)을 찾는 것으로 나타났으며, 보건소 이용에 대해서는 2.8%만이 보건소를 찾는다고 응답하여 저소득층이 보건소를 좀 더 쉽게 편하게 이용할 수 있는 방안을 강구하는 것이 필요할 것으로 판단됨. \n□ 치료를 받기 원하는 기관으로는 개인 병원•의원이 응답자의 절반이 넘는 67.8%가 꼽혔으며 다음으로 종합병원(27.2%)로 나타났다. 보건소의 경우 치료를 받기 원하는 기관으로 응답 한 응답자는 거의 2.2%로 선호가 낮은 것으로 나타났음. \n□ 현재 치료기관과 치료희망기관의 차이가 있는 이유에 대해서 응답자들이 경우 치료비가 부담되기 때문이라는 이유가 48.9%로 나타났으며\'차이가 없다\'라는 응답도 27.8%로 나타났음. \n□ 최근 3년간 보건소 이용자에 대해서 보건소 부분별 만족도를 평가한 결과 대부분의 평가항목에 있어서 보통을 상회했지만, 근접성의 경우 2.9로 나타나 다소 불만족스러운 것으로 나타나 보건소에 대한 접근성을 용이하게 확보하는 방향을 모색해야 할 필요성이 제기 됨. \n3. 저소득층 복지계획\n1)계양구 저소득층 사업현황 및 평가\n□ 2006년 계양구 저소득층 복지 사업 현황\n□ 저소득층 복지부문 평가\n⚬ 저소득층의 복지사업현황을 살펴보면 자활근로부분, 기초생활부분에 예산이 집중되어 있는 것으로 나타남. \n⚬ 자활근로사업이나, 기초생활보장급여는 경직성 경비이기 때문 에 지역사회의 복지 증진을 위한 지방정부의 의지가 반영되기 어려운 구조로서 저소득층에 대한 지방정부의 노력을 경주하고 있다고 보기는 어려움. \n⚬ 그러나 계양구의 저소득층의 실태를 살펴보면, 건강문제가 심각하고, 일자리구하기가 어려운 것으로 나타나고 있어 이 부분에 대한 대책이 강구되어야함. \n2)저소득층 복지 정책목표\n□ 저소득층 복지 정책목표는 "희망을 일구어 나아가는 사회안전망 구축"으로 설정함. \n⚬ 욕구조사결과 생계비지원, 자녀양육비지원, 의료비지원, 주거 환경지원 등이 주요 욕구로 파악되었음. \n⚬ 이에 본 계획에서는 기존의 지원제도를 확대•강화하는 동시에, 저소득층자녀의 교육지원부분에 새로운 프로그램을 개발. \n□ 저소득층의 문제는 단지 저소득의 문제뿐만 아니라 질병, 가정 문제, 장애 등 다양한 사회문제와 연계되어있음. \n⚬ 본 계획에서는 이러한 저소득층 문제를 인식하고 이를 해결 하기 위해 다양한 복지정보를 저소득층에게 제공하고자 함. \n⚬ 저소득층 이외의 차상위계층에 대한 지원은 저소득층 확대로 발생할 수 있는 사회문제를 예방할 수 있기에 본 계획에서는 차상위계층에 대한 지원방안을 제시. \n3)저소독층 복지 중점주진사항\n(1)생활안정지원강화\n□ 생활안정자금 지원 확대\n⚬ 저소득층 주거안정자금(전세금)대출 확대\n- 저소득 무주택자 및 세입자의 주거안정을 위해 저리의 전세자금을 융자 알선. \n- 사업주관부서는 주민생활지원과 자활팀이며 금융기관과 대출 업무 조율. \n⚬ 긴급복지지원사업 확대\n- 계양구청 홈페이지, 사회복지 정보지•소식지 등을 활용한 긴급복지지원사업에 대한 홍보 강화. \n- 사업주관문서는 주민생활지원과 서비스연계팀이며, 복지총괄 기획팀과 홍보 등에 관해서는 연계\n- 사업예산은 국비 80%, 시비10%, 구비10%로 이루어지며, 2006년을 기준(242,914,000)으로 하여 약 10%씩 증액. \n□ 기초생활보호대상자 조사 내실화\n⚬ 통합조사팀 및 서비스연계팀 강화\n- 사업주관부서는 주민생활지원과 서비스연계팀 및 통합조사팀. \n- 원활한 조사활동을 위해 서비스연계팀 및 통합조사팀에 차량 각 1대씩 구입(각 10,000,000원)및 통합조사인력 3인 충원\n- 통합조사팀 및 서비스연계팀의 차량구입비•업무추진비 및 여비 등 포함하여 27,000,000만원 소요. \n□ 보건의료부문과의 연계강화\n⚬ 건강보험체납자등에 대한 생활 실태조사. \n- 저소득층 및 차상위계충에 대해 건강보험체납과 관련하여 실태조사를 실시. \n- 사업기획부서는 주민생활지원과 통합조사팀이며, 동사무소의 사회복지전담공무원의 연계를 통해 실태조사. \n- 보건의료 실태조사 결과 의료급여가 필요한 대상자를 파악하여 차상위의료급여대상자 선정지원. \n⚬ 저소득층 의료복지 강화. \n- 지역 보건소와 연계하여 저소득층에 대한 진료사업 확대. \n- 사업주관부서는 주민생활지원과 서비스연계팀이며, 연계기관 및 부서는 보건소 및 통합조사팀. \n□ 저소득층가정 교육활동 지원\n⚬ 저소득층 자녀 공부방 선생님 지원\n- 공부방에 대졸 미취업생에 대한 공공근로인력 또는 고학력 공익요원, 자원봉사자 등을 배치하여 "학습도우미"제도 운영. \n- 학습도우미제도를 활성화하기 위해 학습도우미로 일할 경우(자원봉사자 한)관내 공공기관(시설관리공단 등)취업에 가산점을 부여하는 방안 고려. \n- 사업주관부서는 주민생활지원과 서비스연계 팀. \n(2)자활사업의 효율성 제고\n□ 자활사업의 범위 확대\n⚬ 차상위 계층의 대한 지원 확대\n- 기초생활보장 비수급자이면서도 빈곤층으로 남아 있는 차상위계층의 자활사업참여를 확대. \n- 주민생활지원과 자활팀에서 사업을 주관하고 자활후견기관과 연계, 예산은 국비 80%, 시비10%, 구비 10%로 이루어지며, 2006년 기준 약 5%씩 증액. \n□ 차상위계층에 대한 자활사업 홍보 강화\n⚬ 계양구 홈페이지를 이용한 지역사회 복지 정보 DB 및 소식지, 정보지 강화. \n- 사회복지 정보에 대한 홍보를 위해 계양구 홈페이지를 강화하여 상시적으로 자활사업정보 제공. \n- 사업주관부서는 주민생활지원과 복지총괄기획 팀에서 담당하고 예산은 계양구 홈페이지 리뉴얼사업(전산개발비)으로 충당. \n(3)지역사회 복지정보시스템 구축\n□ 계양구 홈페이지 강화\n⚬ 복지정보시스템 구축\n- 계양구 홈페이지 개편을 통해 시기별, 대상별, 복지내용별 정보를 수요자 중심으로 한 눈에 볼 수 있도록 구축. \n- 현재 공고 위주로 홈페이지 구현 방식을 벗어나 자신의 정보를 등록하였을 경우 맞춤정보를 제공. \n- 사업기획부서는 주민생활지원과 복지총괄기획팀에서 담당하고 구체적 시스템 구추은 구청 전산팀. \n- 현재 계양구 홈페이지 연동이 가능하도록 설계하고 복지정보 소식지 등의 담당인력 확충이 필요. \n□ 복지정보소식지 강화\n⚬ 웹진 형식의 복지정보소식지 발간\n- 복지정보를 담은 소식지를 웹으로 구현하여 복지 대상자 및 일반주민이 열람할 수 있도록 함. \n- 등록자의 경우, 정보소식지를 메일링 서비스 함. \n- 사업기획부서는 주민생활지원과 복지총괄기획팀에서 담당하고, 기획감사실 전산팀과 연계하여 메일링 서비스 등을 담당 하게 함. \n⚬ 인쇄물 형태의 복지정보소식지 발간(『계양산 메아리』활용)\n- 인터넷에 익숙하지 않거나 접근성이 떨어지는 복지대상자 및 일반주민을 위해, 복지대상자는 일괄발송하고 일반주민의 경우 신청자에 한해 발송. \n- 웹 형식의 복지정보소식지를 가공하여 인쇄물 형태의 소식지 발간. \n4)핵심 추진사업 연차별 계획\n□ 연차별 사업계획\n• 저소득층 분야 추진사업은 생활안정자금 지원 확대•강화, 보건의료부문과 연계강화, 자활사업의 범위 확대, 차상위계층에 대한 자활사업 홍보강화, 계양구 홈페이지 강화, 복지정보소식지 발간으로 선정하였음. \n제7절 노숙인 복지\n1. 노숙인 복지실태 및 현황\n1)건강 상태 실태\n□ 노숙인들에게 자신의 건강상태를 묻는 질문에\'건강한 편\'이라고 60.0%가 응답을 하여 노숙인들의 건강상태가 크게 나쁘지 않은 것으로 나타났음. \n⚬ 또한\'건강하지 못한 편\'15%, \'매우 건강하지 못함\'0.0%로 나타나 절박한 의료서비스가 필요한 대상이 없는 것으로 나타남. \n□ 노숙인들의 의료서비스 경험유무를 묻는 질문에 응답자의 절반이 넘는 11명(55.0%)가 있는 것으로 나타났음. \n⚬ 노숙기간에 의료서비스 경험이 대부분 있는 것으로 나타났기에 의료서비스에 혜택을 전혀 받지 못하는 상태는 아닌 것으로 판단할 수 있음. \n□ 만성질환 여부를 묻는 질문에 대해서 응답자의 절반은\'있다\'나머지 절반은\'없다\'라고 대답했음. \n□ 발병 시기를 묻는 질문에 대해서 병을 갖고 있는 사람들 절반이 노숙생활 이전에 이미 만성질환을 앓고 있는 것으로 나타났으며 노숙생활을 하면서 얻은 사람은 4명(20.0%)로 나타남. \n□ 노숙인들이 앓고 있는 질병으로는 관절염이 8명(40.0%)로 가장 많았고 그 다음으로 고혈압(20.0%)으로 나타났음. \n□ 노숙인들이 주로 진료를 받는 곳은 병원으로 나타났으며, 90.9%인 10명이 병원을 주로 이용하고 있음. \n2)노숙인 생활 실태\n□ 노숙인들은 주로 식사를 무료급식소를 이용하여 해결하는 것으로 나타났음. 응답자의 50.0%인 10명이 무료급식소를 이용하고 그 다음으로 일용직 노동을 통해 식사를 해결하고 있음. \n⚬ 조사결과를 살펴표면, 많은 노숙인들이\'일용직 노동\'이라는 부분이라고 답해, 근로의욕은 있는 것으로 판단되므로\'일자리 소개\'나 취업올 위한 교육 등의 서비스가 필요함. \n□ 겨울에 노숙인들이 주로 잠을 청하는 곳으로 지하철 역사가 7명(35.0%)로 가장 많았으며 그 다음으로 사우나가 5명으로 나타났음. 노숙인 쉼터를 이용한다고 대답한 사람은 전체 응답자의 5.0%인 1명으로 나타나 노숙인 쉼터에 대한 홍보 및 접근성을 확보할 필요성이 제기됨. \n⚬ 겨울철에는 노숙자 쉼터 입소자가 늘어나고, 봄부터 가을까지는 노숙자 쉼터의 입소자가 줄어드는 것을 감안하면 추위가 노숙자 생활에 가장 어려운 점인 것으로 보임. \n□ 노숙생활에 가장 어려운 점으로 식사문제를 전체 응답자의 55.0%인 11명이 응답했으며 다음으로 잠자리가 20.0%로 나타남. \n□ 노숙인들의 주민등록 말소자현황을 살펴보면 대부분의 응답자인 14명(70.0%)이\'그렇다\'라고 응답해 노숙인의 경우 주민등록이 말소된 것으로 파악됨. \n□ 주민등록 말소로 인한 불이익에 대해서 전체 응답자의 7명(35.0%)이\'그렇다\'라고 응답하였고 나머지 13명(65.0%)는\'불이익이 없다\'라고 응답. \n3)노숙인 쉼터 관련 실태\n□ 노숙인 쉼터 만족도에 대한 평가에서 만족을 표시한 응답자는 55.0%인 11명이 응답을 하였다. 평균 3.5로 보통 이상의 응답을 하였음. \n□ 노숙인 쉼터 입소 희망 정도가 어느 정도인지에 대해서는\'잘모르겠음\'이 11명(55.0)으로 노숙인 쉼터 입소에 대한 정밀한 조사가 이루어져야 할 것으로 판단됨. \n□ 노숙인 쉼터 입소거부 이유의 다수가 생활에 대한 제약이 가장 큰 이유로 85.0%인 17명이 응답함. \n□ 노숙인 쉼터 활성화를 위해 필요한 사항을 묻는 질문에 응답자의 대부분의 평가가 필요성을 높게 판단하고 있는 것으로 나타났음. \n⚬ 노숙인 쉼터의 물리적 환경개선뿐만 아니라 자활프로그램의 다양성에 대한 필요성과 다른 기관과의 연계에 대해서도 그 필요성을 보이고 있음. \n2. 노숙인 복지서비스 욕구\n□ 노숙인의 경우 가장 필요한 프로그램으로 생활안정자금을 지원해주는 것을 꼽았으며(10명, 50.0%), 다음으로 숙식해결(20.0%)과 무료직업훈련(20.05)으로 응답함. \n□ 노숙인이 필요로 하는 서비스에 대한 필요성에 대해서 모든 항목에 대해서 필요하다고 응답하고 있는 것으로 나타났으며, 무료급식소(4.5), 무료진료소(4.4)등의 순으로 나타났음. \n□ 노숙인을 위한 계양구 역점사업 순위로는 응답자의 40.0%인 8명이 노숙인 쉼터 확충을 제일 많이 응답하였으며 그 다음으로 취업정보•취업기술교육제공(30.0%)로 조사됨. \n3. 노숙인 복지계획\n1)계양구 노숙인 복지 사업현황 및 평가\n□ 2006년 계양구 노숙인•부랑자 복지 사업 현황\n⚬ 계양구는 현재 인천에서 유일하게 노숙자 관련 복지시설을 운영하고 있음. \n⚬ 노숙인•부랑자 관련 복지사업은 복지시설운영비에 쓰이고 있고 일부예산은 행려 사망자와 관련되어 정책되어 있음. \n□ 노숙인 복지부문 평가\n⚬ 노숙인들의 삶에 가장 중요한 복지욕구가 의식주문제 해결로 나타나고 있어 이 부분에 대한 해소방안 마련되어야 함. \n⚬ 일부 노숙인의 경우 사회복귀를 거부하는 경향이 있으나 대다수 노숙인의 경우 사회복귀를 간절히 원하고 있어 이들에 대한 지원이 필요함\n⚬ 계양구의 노숙자•부랑인 관련 복지사업현황을 보면 인천광역시의 다른 구와는 달리 노숙인 수용시설이 있어 노숙인 복지에 있어 선진적이라고 평가할 수 있지만 사회복귀에 실질적으로 도움이 될 수 있는 공동가정생활제도가 필요함. \n2)노숙인 복지 정책목표\n□ 노숙인 복지 정책목표는 "사회로의 복귀를 가능하게 하는 따뜻한 계양"으로 설정함. \n⚬ 욕구조사결과 취업, 의료서비스에 대한 욕구가 높은 것으로 조사되었음. \n⚬ 이에 본 계획에서는 주요정책부분을 사회복지지원강화, 기초 생활 지원 서비스 강화로 설정함. \n3)노숙인 복지 중점추진사항\n(1)사회복귀 지원 강화\n□ 취업지원 강화\n⚬ 취업교육 실시\n- 종합사회복지관 또는 자활후견기관 등을 통한 취업 교육 프로그램을 마련하여 사회복귀 지원. \n- 사업주관은 주민생활지원과 복지총괄기획팀에서 담당하고 연계부서는 노동복지회관 및 종합사회복지관, 자활후견기관 등 과 연계하여 실시하고, 사업예산은 국비 지원을 받아 충당. \n• 취업정보 제공\n- 계양구청 내 취업알선센터를 활용한 취업 정보 제공. \n- 사업주관부서는 주민생활지원과 복지총괄기획팀이며 연계기관은 취업알선 센터. \n□ 법률지원 서비스 제공\n⚬ 법률 상담 지원\n- 개인회생제도 및 파산제도 등의 신용불량자 구제제도와 민법상의 다양한 법률 내용을 상담할 수 있는 이동 법률 상담 프로그램을 제공하여 월 단위 또는 격월단위 방문 상담. \n- 사업주관부서는 복지총괄기획팀 이며 사업 예산은 비 예산. \n□ 사회복귀를 위한 지원시설 강화\n⚬ 공동생활 가정(Group Home)제도 시행\n- 사회복귀가 당장은 불가능하지만, 의지가 강하고 여건이 충분한 노숙인의 경우, 노숙자 수용시설보다는 공동생활가정을 통하는 것이 사회복귀에 긍정적 효과가 있는 것으로 판단 됨. \n- 이에 남녀 1개소(34명 수용)씩 2개소의 사회복귀를 위한 공동생활가정제도를 두어 사회복귀를 지원. \n- 다만, 거주기간 및 입소조건 등을 명확히 규정하여 합리적 운영이 가능하도록 함. \n- 사업기획부서는 주민생활지원과 총괄기획팀이며, 사업시행은 민간기관에 위탁하여 시행. \n- 사업예산은 임대(전세)료 10,000,000만원X2개소와 사회복지사 1명 고용(15,000,000원)이며 시비로 충당. \n(2)기초생활 지원서비스 강화\n□ 생필품 지원\n⚬ 개인 위생용품 무료 지원\n- 치약, 칫솔, 비누, 상비약 등 개인위생용품 지원. \n- 사업기획부서는 주민생활지원과 총괄기획팀이며, 연계기관은 보건소 및 자원봉사센터 등. \n- 겨울철을 대비하여 중고 담요, 겨울옷 등을 사회복지기관 및 단체 등의 기부를 받아 제공. \n- 사업기획부서는 주민생활지원과 총괄기획 팀이며, 연계기관은 자원봉사센터 및 민간복지기관 등. \n□ 보건의료서비스 강화\n⚬ 방문진료사업 실시\n- 노숙자 쉼터에 보건소의 방문보건팀과 협의하여 정기적인 진료서비스 및 간단한 처치를 제공. \n- 사업기획부서는 주민생활지원과 복지총괄기획이며, 연계기관은 보건소. \n4)핵심 추진사업 연차별 계획\n□ 연차별 사업계획\n- 노숙인 분야 추진사업은 취업지원 강화, 법률지원 서비스 제공, 사회복귀를 위한 지원시설 강화, 생필품 지원, 보건의료서비스 강화로 선정함. \n제4장 지역사회복지 기반조성계획\n제1절 사회복지 전달체계\n1. 지역사회복지협의체\n1)지역사회복지협의체 개요\n□ 지역사회복지협의체는 사회복지에 관한 중요사항과 지역사회 복지계획을 심의 또는 건의하고, 사회복지•보건의료관련 기관•단체가 제공하는 사회복지서비스강화 및 보건의료서비스 연계•협력을 강화하기 위하여 협의체 상호간에 정보를 공유하고 운영의 효율성을 기하고자 구성. \n□ 사회복지사업법 제4조 및 법 제7조의2를 통한 지역사회복지협의체의 그 설치근거를 살펴보면 다음과 같음. \n⚬ 사회복지사업법은 관내의 사회복지사업에 관한 중요사항과 지역사회복지계획을 심의 또는 건의하고 사회복지서비스 및 보건의료서비스의 연계협력을 강화하기 위하여 지역사회 복지협의체를 설치할 근거를 제시. \n□ 계양구 지역사회복지협의체 운영조례를 살펴보면 지역사회복지협의체와 관련된 기능을 다음과 같이 제시하고 있음. \n⚬ 지역사회복지협의체(이하 "대표협의체"라 한다)는 다음 각 호의 사항을 심의•건의하거나 인천광역시계양구청장(이하 "구청장"이라 한다)의 자문에 따름. \n⚬ 지역사회복지계획의 수립, 시행, 평가에 관한 사항\n⚬ 사회복지서비스 및 보건의료서비스의 연계•협력에 필요한 사항. \n⚬ 지역주민의 복지욕구조사, 지역 내 복지자원 조사 및 개발에\n관한 사항. \n⚬ 구청장이 지역복지시책 추진을 위해 필요하다고 인정하는 사항. \n⚬ 그 밖에 지역사회 복지사업 전반에 대한 협의 및 건의 등. \n⚬ 대표협의체는 인천광역시계양구의 사회복지서비스와 보건의료서비스 등을 제공하는 관련기관•단체간의 연계 협력 업무 를 수행함. \n⚬ 실무협의체는 제1항 및 제2항의 규정에 의한 대표협의체의 기능을 지원함. \n□ 계양구의 지역사회복지협의체 구성을 살펴보면 우선 대표협의체는 당연직3인, 주민대표2인(구의원), 수요자대표3인, 사회복지관련시설6인, 의료단체3인, 기타2인으로 구성되어 있으며, 실 무협의체는 대표협의체(위원장, 당연직)1인, 당연직(서비스연계, 보건행정담당)2인, 사회복지실무자1인, 공익단체2인, 의료시설1인, 복지위원1인, 보건의료실무자3인, 사회복지이용시설7인, 기타2인으로 구성되어 있음. \n□ 계양구 지역사회복지협의체 실무분과위원회는 아래 표와 같이 통합서비스지원분과, 여성•아동분과, 생활보장분과, 보건의료분과, 노인•장애인분과로 구성되어 있으며 분과의 기능은 다음과 같음. \n⚬ 사례회의를 통한 지역사회 내 복지욕구 및 서비스 공급자 관련 정보공유. \n⚬ 지역사회 내 복지욕구와 서비스 공급자간 복지자원 연계•지원방안 협의 결정. \n⚬ 지역사회 내 복지서비스 및 보건의료서비스 간 공동사업의 추진 및 운영. \n□ 계양구의 사회복지협의체관련 예산현황을 살펴보면, 회의참석수당(대표•실무자협의체 각3회)이외의 다른 내용은 포함되어 있지 않음. \n□ 계양구 지역사회복지협의체 예산현황은 다음과 같음. \n2)지역사회복지협의체 발전 방향\n□ 지역사회복지협의체의 위상 확립\n• 지역사회복지협의체는 현재 구성된 지 얼마 되지 않아 법률\n에서 규정하고 있는 위상을 갖추고 있지 못함. \n⚬ 지역사회복지협의체의 협의내용이 사회복지사업의 협의내용이 될 수 있도록 협의체의 위상정립이 필요함. \n⚬ 지역사회복지협의체의 위상이 확립되기 위해서는 지역사회복지계획 수립 및 변경의 실질적 권한이 행사될 수 있어야함. \n⚬ 다만, 권한의 행사가 상위법령 및 신의성실의 원칙에 위배되지 않도록 하는 조례 및 규칙이 세부적으로 마련되어야함. \n□ 지역사회복지협의체의 역할 제고\n⚬ 지역사회복지협의체의 역할이 원활이 수행될 수 있도록, 협의체의 구성원간의 의사가 교환될 수 있는 여건 마련. \n° 대표협의체 및 실무협의체의 의사가 원활이 교환될 수 있도록 협의체의 전 구성원이 참석할 수 있는 간담회나 워크숍을 개최하여 지역사회복지현안에 대한 의견 교환 및 정보공유. \n□ 지역사회복시협의체 활성화를 위한 사업계획\n□ 전체 구성원 참석 워크숍\n• 대표협의체, 실무협의체, 실무분과 구성원 전체가 참석하여 지\n사회 복지증진 및 복지전달체계 개선을 위한 워크숍 개최\n⚬ 사업주관부서는 주민생활지원과 서비스연계 팀이며 사업예산은 구비로 충당. \n□ 지역사회복지협의체회의 및 실무협의체회의 개최\n⚬ 지역사회복지계획의 수립, 시행, 평가에 관한 사항 미 사회복지서비스 및 보건의료서비스의 연계•협력에 필요한 사항 등을 논의하고 지역사회복지사업 전단에 대한 협의가 원활이 진행될 수 있도록 회의 개최. \n⚬ 사업주관부서는 주민생활지원과 서비스연계팀이며 사업예산은 구비로 충당. \n□ 지역사회복지협의체 운영 간사 확보\n⚬ 지역사회복지협의체의 원활한 운영을 통하여 협의체가 지역사회복지서비스 전달•연계체계의 중심이 될 수 있도록 운영 간사를 확보해야함. \n⚬ 사업주관부서는 주민생활과 서비스 연계팀이며 사업예산은 구비로 충당. \n2. 지역인력자원 발굴\n1)자원봉사 실태 및 현황\n(1)자원봉사인원 현황\n□ 계양구 자원봉사인원 현황은 아래와 같음. 가장 많은 자원봉사 분야는 사회복지분야(3,115명), 교통•질서분야(1,590명), 노인분야(1,572명)로 나타났음. \n□ 자원봉사활동 세분류별 인원은 아래와 같음. 봉사활동 인원을 살펴보면 기타(3,233명), 교통질서(1,590), 가시지원(763명), 목욕 봉사(735명)순으로 나타났음. \n(2)자원봉사 실태\n□ 현재 자원봉사를 한적이 있거나 하고 있느냐는 질문에 대해서 일반주민의 82.7%가 부정적인 대답을 한 것으로 나타나 주민들이 적극적으로 자원봉사에 참여할 수 있도록 하는 유인책을 마련하는 것이 필요함. \n□자원봉사활동에 대한 참여방법 중 각종단체를 통한 봉사활동참여를 가장 선호하는 것으로 나타났으며 다음으로 사회복지시설•단체자원봉사활동으로 나타났음. \n□ 현재 기부를 하고 있다면 월 기부금액에 대해서 응답자의 53.3%가 1천원(53.3%)으로 가장 많은 비율을 차지하고 있으며 2천원(6.7%), 3천원(6.7)로 나타났음. \n□ 관내 기부대상이 있는지에 대해서 응답자의 30.8%가\'예\'로 대답했으며 46.2%가 계양구 내가 아닌 다른 곳에 기부하는 것으로 나타났음. \n⚬ 그런데 여기서 주목해야할 조사결과는 전체 370명이 대상자중에서 13명이라는 아주 소수의 사람들만이 기부활동을 하고 있는 것으로 조사됨. \n2)자원봉사 활성화 방안\n(1)자원봉사활동 활성화 장애요인\n□ 월간 적정 봉사활동 횟수를 묻는 질문에 대하여 응답자의 65.7%가 1회라고 답을 하였으며 16.8%가 2회라고 응답. \n□ 연간 적정 기부금액의 규모에 대해서 49.7%가\'5만원이하\'로 응답하였으며\'6-10만원\'이 22.7%로 나타났음. 평균 연간기부금액인 13만 6천원으로 나타났음. \n⚬ 그러나 앞서 제시된 기부실태조사 내용을 살펴보면, 월별 평균 1천 3백원을 기부하고 있어, 실제 1년으로 환산할 경우 1만5천원 가량 기부하고 있는 것으로 나타나 적정치라고 예상한 13만 6천원과는 많은 차이가 있음. \n⚬ 즉, 많은 사람들이 봉사활동이나 기부활동을 위해서는 많은 시간과 금전을 할애해야 하는 것으로 인식하고 있음. \n□ 봉사활동을 하지 않는 사람들에게 왜 자원봉사활동을 꺼려하는지에 대해 물어본 결과\'시간적 여유가 없어서\'(50.3%), \'금전적 여유가 없어서\'(28.6%), \'봉사방법의 정보가 없어서\'(10.1%)로 나타났음. \n(2)활성화 방안 모색\n□ 자원봉사참여방법을 아는지에 대한 질문에 대해 살펴보면, 대부분의 응답자들이 각각의 방법에 대해서 복지시설자원봉사(65.0%), 복지시설에 대한 후원(60.5%), 개인적 후원(68.0%), 각 종 단체를 통한 봉사(62.7%)를 아는 것으로 나타났음. \n□ 앞으로 자원봉사활동에 대한 참여 여부를 묻는 질문에 대해 35.1%만이 참여의사를 밝혔고, 64.9%는 부정적으로 응답한 것으로 나타났음. 동별로 보면 작전서운동(78. 8%), 효성동(69,5%), 작전동(65.9%)순으로 긍정적인 응답을 하였음. \n□ 자원봉사활동을 하고 있지 않은 사람들에게 만약 참여를 하게 된다면 어떤 방법으로 참여할 것인가에 대해서 33.3%가 사회 복지시설•단체자원봉사활동을 선호하는 것으로 나타났음. \n□ 자원봉사를 활용화시키기 위해서는\n⚬ 첫째, 적은 시간과 돈으로도 얼마든지 자원봉사를 할 수 있도록 자원봉사 참여제도의 변화가 필요하고, \n⚬ 둘째, 다양한 참여방법으로 자원봉사를 할 수 있도록 여건을 마련해야 할 것이며, \n⚬ 셋째, 기관 참여방법 및 봉사활동 종류를 다양하게 개발하여 많은 단체나 기관이 참여할 수 있도록 해야 할 것임. \n(3)활성화 방안 제시\n□ 민간기관•단체와 복지시설 간 자매결연 사업\n⚬ 계양구내의 민간기관•단체와 복지시설 간 자매결연 사업을 통해 민간기관의 자원봉사활동 참여를 유도하고 이를 지원하여 봉사활동을 활성화시킴. \n⚬ 사업기획부서는 서비스연계 팀이며 연계기관은 자원봉사센터로 함. \n⚬ 자발적 봉사단체 지원\n⚬ 지역 내 다양한 사회단체(계, 친목단체, 취미모임 등)가 봉사 활동에 참여할 수 있도록 정보 제공. \n⚬ 봉사활동 참여 시 교통비 차원의 실비 제공. \n⚬ 사업기획부서는 사업주관부서는 복지서비스과 여성•아동복지 팀으로 하고, 자원봉사센터와 연계하여 시행하고, 사업예산은 자원봉사센터 확대사업에서 충당\n□ 자원봉사 홍보활동 강화\n⚬ 계양구는 자원봉사 소식지 「다릿돌」을 발간하여 홍보사업을 진행하고 있지만 앞의 자원봉사자 설문조사 결과 후원방법이나 참여방법을 모르는 주민들이 있으므로 자원봉사와 관련된 홍보사업의 효과가 낮은 것으로 판단됨. 이에 홍보활동을 강화가 필요. \n⚬ 예산은 홍보전단지 제작 등의 비용이 소요되고 이를 구비에서 충당하고 홍보활동 인력은 자원봉사자를 활용. \n⚬ 사업주관은 자원봉사센터이며 연계부서는 주민생활지원과 복지 총괄기획팀\n□ 가족봉사프로그램 확대\n⚬ 가족의 해체와 지역사회문제를 예방하기 위하여 가족공동의 봉사활동으로 유도할 수 있는 프로그램을 개발하여 가족단위의 봉사활동 활성화 유도. 여성복지부문의 가족봉사프로그램과 동 일한 사업임. \n⚬ 사업기획부서는 주민생활지원과 서비스연계 팀이며, 자원봉사센터에 위탁하여 시행. \n⚬ 사업예산은 현재 2개 단체 400만원이 지원되고 있고, 연차적으로 2010년까지 50개 단체를 목표로 하고 예산은 구비. \n□ 도•농 연계 프로그램 강화\n⚬ 도•농복합지역인 계양구의 특성을 살려 도시지역의 농촌지역에 대한 봉사활동 프로그램 개발. \n⚬ 특히, 농촌지역은 노령인구의 증가로 인해 노동력의 부족이 심각하므로 생태체험과 동시에 진행할 수 있는 봉사활동 프로그램 개발\n⚬ 본 사업은 아동•청소년 복지사업과 동일한 프로그램으로서 사업주관부서는 주민생활지원과 서비스연계 팀에서 기획하여 집행하고 연계기관은 자원봉사센터임. \n□ 자원봉사센터 운영지원 강화\n⚬ 자원봉사 활성화를 위하여 2명의 전담인력 확충과 다양한 자원봉사 프로그램 개발이 요청됨. \n⚬ 이를 위하여 자원봉사센터에 대한 예산지원이 필요하고, 예산 지원에 따른 자원봉사센터의 책임성 제고. \n⚬ 사업기획부서는 서비스연계팀이며 인력확충 및 운영은 자원봉사센터에 위탁하여 집행. \n3. 지역복지 네트워크 연결방안\n1)네트워크 연계 모델\n□ 지역복지 네트워크 구성\n⚬ 계양구 사회복지 전달체계는 공공부문과 민간부문의 자원을 공유하면서 가장 효율적인 자원배분체계를 갖추어야 함. \n⚬ 효율적인 자원배분체계는 완전정보상태에서 이루어질 수 있으므로 복지공급자 및 수요자들에 대한 정확한 파악이 절대적인 조건이라고 할 수 있음. \n⚬ 이를 위해서는, 복지욕구조사 및 지역사회복지자원에 대한 정확한 파악과 이를 가능하게 하는 복지공급기관간의 밀접한 연계가 필요. \n• 지역복지 네트워크란 공공부문 및 민간부문의 복지공급자간의 협동적이며 효율적인 연계체계. \n2)연계방안 제시\n□ 지역주민의 복지수요에 민감하게 반응할 수 있도록 하는 모델로서 지역사회복지를 공급하는 다양한 주체들이 지역사회복지협의체로 통합되어야 함. \n⚬ 첫째, 지역주민의 복지수요에 대한 파악이 필요함. \n⚬ 둘째, 각 주체별 복지대상자에 대한 정보 공유 및 이를 가능하게 하는 협력체계 구축. \n⚬ 셋째, 종교기관, 산업체 등과 같이 지역 내 모든 복지자원을 활용해야 함. \n□ 지역사회복지협의체를 통한 복지서비스 연계극대화\n⚬ 지역사회복지협의체의 정보기능 강화 필요. \n⚬ 지역사회복지계획 및 기타 지역복지현안에 대한 자문 및 심의 권한 확대. \n⚬ 공공기관은 직접생산보다는 키잡이로서의 역할만을 하고 복지서비스의 집행은 사회복지협의체를 중심으로 복지공급자 사슬에 의해 실행. \n□ 복지 공급의 효율성 제고를 위한 지역사회복지협의체\n⚬ 복지서비스의 중복투자를 최소화하여 효율적 복지자원 배분을 지역사회복지협의체가 중심이 되어 계획. \n⚬ 복지공급의 연계의 성과를 정기적으로 평가하여 그 결과를 반영하여 연계강화를 추구. \n⚬ 평가를 바탕으로 우수한 연계사례에 대해서는 인센티브를 제 공하여 지속적인 복지서비스 공급의 연계 강화. \n제2절 보건 의료 연계방안\n1. 보건의료 욕구조사 결과\n1)보건의료 실태\n□ 응답자들에게 월 평균 의료비의 지출 규모를 알아본 결과는 아래의<표IV-18>에서 나타난 바와 같이 장애인 응답군이 10만 1천 7백원으로 가장 높음. 장애인을 제외한 나머지 응답군의 경우, 최소 2만 7천 9백원(아동)에서 최대 3만 7천 1백원(노인)사이의 분포를 보임. \n□ 응답자들이 아플 경우 주로 이용하는 치료방법(기관)에 대해 조사한 결과, 장애인을 제외한 모든 응답군에서 병•의원을 가장 많이 이용하는 것으로 나타났으며, 그 다음으로는 약국을 이용한다는 응답이 많았음. 장애인의 경우는 일반병원. \n⚬ 장애인의 경우는 일반병원(78.1%)을 가장 많이 이용하며, 재활전문기관•한방 병•의원(6.3%), 장애인 복지기관(47%)등의 순으로 응답 함. \n□ 응답자들이 아플 경우 치료받거나 이용하고 싶은 의료기관에 대한 응답으로는, 일반주민 응답군에서는 대형병원(76.2)이 가장 많고 그 다음으로 보건소(17.3%)순으로 나타남. \n⚬ 하지만 나머지 보육•아동•노인•여성•저소득층 응답군에서는 개인 병•의원이 가장 많은 응답결과를 나타내었고, 그 다음으로 대형병원이 높은 응답결과를 보임. \n2)보건의료기관\n□ 희망 의료기관을 달리하여 조사한 장애인 응답군에서는 일반 병원(45.3)이 가장 높은 응답결과를 나타냄. 그 외 재활전문기관(28.1%), 한방병•의원(18.8%)등의 순서로 조사됨. \n□ 응답자들의 현재 치료받고 있는 의료기관과 희망 의료기관이 차이나는 이유는, 일반주민과 보육, 여성 응답군은\'차이가 없다\'(25.9%, 35.7%, 32.1%)가 가장 많았으며, 아동 응답군은\'병이 크지 않아서\'(41.9%), 노인과 저소득층 응답군은\'치료비 부담\'(32.0%, 48.9%)등이 가장 큰 이유인 것으로 나타남. \n□ 보건소 이용자들에게 보건소의 부분별 만족도를 조사한 결과, 대부분의 평가항목에 있어서 보통을 상회하였지만, 저소득층 응답군의 경우\'거리(근접성)\'에 대한 만족도가 다소 불만족스러운 것으로 나타나(2.9)보건소에 대한 접근성을 용이하게 확보하는 방향을 모색해야 할 필요성이 제기 됨. \n2. 보건•의료 전달체계\n1)보건•의료분야 정책목표\n□ 보건•의료분야 정책목표는 "복지기관과의 연계를 통한 계양구민의 삶의 질 제고", "찾아 움직이는 보건의료서비스"로 설정. \n⚬ 욕구조사결과 취약계충들의 의료서비스에 대한 욕구가 높은 것으로 조사되었고, 그들의 건강이 양호하지 않으며, 특히 장애인의 경우 한방재활에 높은 욕구를 나타냈음. \n⚬ 지역보건소에 대한 지역주민의 평가를 살펴보면 전체적인 만족도는 양호하나 진료과목에 대한 항목은 다른 항목에 비해 낮은 만족도를 보이고 있고 가장 큰 문제는 많은 사람들이 보건소를 이용하지 않는 것으로 나타남. \n⚬ 이에 본 계획어서는 보건•의료분야 주요정책부분을 요보호대상자 의료서비스 강화, 현황진단을 통한 효율적 시설확충, 복지네트워크와의 연계강화로 설정함. \n2)지역보건•의료분야 중점추진사항\n(1)요보호대상자 의료서비스 강화\n□ 취약계층 방문보건사업 강화\n⚬ 장애인, 소년•소녀가장, 노인에 대한 방문진료사업\n- 장애인, 노인 중 재가복지서비스가 필요한 주민들에게는 방문 진료서비스가 매우 필요하며 소년- 소녀가장의 경우 재가서비스 대상은 아닐지라도 보건의료 대상에 있어서 사각지대에 놓일 상황이 높음으로 이들에 대한 방문보건진료사업도 필요. \n- 사업주관부서는 주민생활지원과 서비스연계 팀이며, 연계부서는 보건소 방문보건팀. \n□ 질병예방사업 강화\n⚬ 관내 보육•교육시설 및 복지기관에 방문 예방교육\n- 유아 및 초등학교 저학년을 대상으로 한 구강보건에 대한 방문교육을 실시. \n- 노인 및 일반인을 대상으로 복지기관을 활용하여 환절기 건강관리법 등을 방문교육함. \n- 사업주관부서는 주민생활지원과 서비스연계 팀이며, 연계부서는 보건소의 건강증진팀. \n□ 한방재활사업 확충\n⚬ 장애인 한방재활사업 실시\n- 장애인에 대한 욕구조사 결과 한방재활의료에 대한 수요가 높게 나타나 이에 대한 대책이 필요함. \n- 보건소에서는 한방재활전문인력을 확충하여 장애인들의 복지 욕구를 해소할 필요가 있음. \n- 사업주관부서는 보건소 보건행정팀이며 연계부서는 복지서비스과 장애인 복지팀. \n(2)현황진단을 통한 시설확충\n□ 실태조사를 통한 의료시설 확충\n⚬ 보건의료복지 실태조사\n- 보건의료복지와 관련하여 실태조사를 통해 지역의 수요 및 현황을 예측•파악하여 지역의 적실성 있는 의료시설 구축. \n- 사업주관부서 및 사업예산은 "건강보험체납자등에 대한 생활 실태조사" 사업과 동일함. \n- 실태조사 후 차상위 계층에 대한 의료보험료 지급. \n□ 보건소 이용률 및 만족도 제고\n⚬ 보건소 만족도 모니터링\n- 지역주민을 통한 보건소 모니터링을 실시하고 보건소를 이용하는 주민들의 만족요인과 불편사항을 파악하고 이용하지 않는 주민들의 불이용 이유를 파악하여 만족도 및 이용률 제고. \n- 사업주관부선 및 사업예산은 "건강보험체납자등에 대한 생활 실태조사" 사업과 동일함. \n(3)복지네트워크와의 연계강화\n□ 지역민간의료기관과 협조체계 강화\n⚬ 질병치료 및 예방교육 실시\n- 보건소의 각 분야별 전문의가 부족한 상황이므로 관내 병원의 전문의를 활용하여 방문교육이나 관내 병원에서 치료 및 예방 에 관한 교육을 실시. \n⚬ 의료데이터베이스 구축\n- 보건소에서 치료할 수 없는 중증 질병이나 위급상황에 대처하기 위해 병상이나 전문의에 대한 의료데이터베이스 구축•활용. \n- 사업주관부서는 보건소 보건행정팀이며 연계부서는 주민생활 지원과 서비스 연계팀\n□ 지역사회복지 향상을 위한 의료서비스와의 협력체계 강화\n⚬ 가정방문도우미와의 연계강화\n- 복지사업 분야에서의 가정방문도우미는 재가복지서비스의 공급을 책임지고 있어 거동이 불편한 요보호대상자들의 상황을 파악하기 용이. \n- 이들과 보건소와의 연계를 통해 취약계층에 대한 보건•의료서비스의 효율성 제고. \n- 사업주관부서는 서비스연계팀이며, 연계부서 및 기관은 보건소 방문 보건팀 및 재가복지센터. \n⚬ 보건소 내 사회복지사 충원\n- 사회복지와 보건의료서비스와의 협력체계를 강화하고, 지역내 유관기관과의 연계업무를 담당할 사회복지사 충원. \n3)복지분야 연계 사업 연차별 계획\n□ 연차별 사업계획\n⚬ 보건의료서비스 분야 추진사업은 취약계층 방문보건사업 강화, 질병예방사업 강화, 한방재활사업 확충, 실태조사를 통한 의료서비스 개발, 보건소 이용률 및 만족도 제고, 지역민간의료기관과 협조체계 강화, 지역사회복지 향상을 위한 의료서비스와의 협력체계 강화를 선정함. \n4)지역복지•보건의료 연계\n□ 지역복지•보건의료 연계 모형\n⚬ 지역복지•및 보건의료분야와의 연계는 계양구청, 지역사회복지협의체, 보건소가 보건•복지 중심체의 역할을 하고, 그밖의 지역사회의 민간단체 및 조직, 민간의료기관, 사회복지기관, 자원봉사센터, 인천광역시 등과 상호 유기적인 연계속에서 지역사회의 복지 및 보건•의료분야의 서비스를 전달. \n제3절 행•재정 계획\n1. 사회복지 행정조직 개편\n1)계양구 본청 사회복지 행정조직 개편\n□ 계양구 사회복지관련 부서의 개편배경은 다음과 같음. \n⚬ 사회경제적 환경변화에 따른 복지수요와 정책의 지속적 확대\n⚬ 행정수요의 다원화로 종합적인 서비스제공체계 구축 필요\n⚬ 주민생활지원 서비스의 행정체계의 수요자 중심으로 개편\n□ 계양구 주민생활지원관련 조직의 개편방향은 다음과 같음. \n⚬ 구 본청의 주민생활지원 조직 통합 및 신설\n- 복지, 고용, 여성, 보육, 평생교육, 문화체여가지원 등 기능강화\n⚬ 동사무소 주민생활지원 기능 중심으로 개편\n□ 계양구는 2실 3국 14과 79팀의 1실 3국 16과 81팀으로 조직개편을 시행하였음. 본 계획에서는 개편된 조직 중 사회복지관련 조직현황을 살펴보도록 하겠음. \n□ 기존 사회산업국을 주민생활지원국으로 명칭을 변경하고 사회 산업국 산하의 사회복지과를 주민생활지원국으로 명칭을 변경\n⚬ 사회복지과의 전기능을 신설되는 주민생활지원과와 복지서비스과로 업무 재조정하여 동사무소 사회복지 업무 이관사무를 포함하여 기능별팀 신설 및 명칭변경 하여 업무 재배치함. \n⚬ 주민생활지원국은 주민생활지원과, 복지서비스과, 문화공보과, 환경위생과, 청소행정과, 지역경제과로 구성되어 있으며 복지 관련 업무는 주민생활지원과와 복지서비스과에서 담당. \n2)계양구 동사무소 조직 개편\n□ 기구 조정\n⚬ 현정원 수준 유지\n⚬ 행정직 전환배치(주민생활지원담당)\n⚬ 사회복지직 3명은 2명만 존치(1명은 구로 배치)1명인 동은 현 수준유지\n⚬ 기능직은 1명(운전)만 존치하고 일반직으로 충원\n□ 계양구는 주민생활지원을 위하여 다음과 같이 동 기능을 조정 하였음. \n⚬ 동사무소 주민생활지원 기능 중심으로 개편\n- 일반행정•민원중심에서 주민생활지원기능 중심으로개편. \n- 현장방문, 심충상담, 정보제공, 관련기관 의뢰 • 연결로 현장 성, 접근성 기능 강화. \n- 동정원은 현 수준 유지 및 직급조정(단, 계양1동, 계양2동은 1명씩 증원 하고, 효성1동은 조무직렬1명을 행정직렬로 대체). \n4)인력 수요 추정\n□ 추가인력 수요 추정의 원칙\n⚬ 추가소요인력 중 민간에 위탁하여 운영해야 할 복지기관 및 시설은 인력수요에 배제하고 기관운영비에 포함하였음. \n⚬ 본 계획의 인력수요 추정은 구에서 직영하는 시설 및 기관의 인력수요만을 추정한 것임. \n⚬ 각 기관 및 시설의 재정계획에는 인건비를 반영하여 재정계획을 수립하였음. \n□ 비공무원 추가인력 수요\n⚬ 보육복지\n- 보육복지부문에서는 "지역사회보육상담실 운영지원 확대"사업에 상담인력 1인 확충. \n⚬ 아동 청소년 복지\n- 아동 청소년 복지사업에서는 "지역아동지원센터 운영 강화"를 위해서 1인의 상담인력 필요. \n⚬ 노숙인 복지\n- 노숙인 복지에는 "공동가정생활제도"사업을 위해서 1인의 사회복지사가 필요. \n□ 공무원 추가인력 수요\n⚬ 노인복지 부문의 사업강화 및 신설되는 시설관리를 위해서 1명의 사회복지전담 공무원 필요. \n- 보건소 방문보건팀 추가 1개팀 필요. \n- "방문간호사" 2명의 추가 인력이 필요한 것으로 추정. \n⚬ 장애인 복지\n- 장애인 복지부문에서는 "재가 복지 및 의료서비스 확대"사업 에서 보건소의 방문보건팀을 1개팀 확충할 필요가 있음. \n- "방문간호사 1인의 인력수요가 필요함. \n⚬ 지역사회복지협의체 및 자원봉사활동 강화 등을 위한 서비스연계팀에 1명의 사회복지전담공무원 필요. \n⚬ 저소득층 복지\n- 저소득층 중 취약계층에 대한 의료서비스 강화를 위해서 0.5팀(1명)의 방문간호사 필요. \n- 통합조사팀에 조사인력 3인 추가 소요. \n- 조사량에 비해 조사요원이 부족. \n⚬ 보건소\n- 보건소내 사회복지사 1인 배치\n- 지역내 유관기관 협력 및 보건복지연계 담당. \n2. 재정 계획\n1)재정 수요 추정 원칙\n□ 본 계획에서의 재정수요 추정은 계양구의 낮은 지방재정자립도를 고려하여 기존사업 중 핵심추진사업은 연차적으로 약 10% 증가시켰고, \n□ 핵심추진 사업이 아닐 경우는 연차적으로 인건비 5%, 인건비를 제외한 사업경비는 3%로 증가시켰음. \n□ 신규사업은 복지수요를 고려하여 편성한 것으로서 연차적으로 5〜30%를 증가 시켰음. \n□ 본 계획에서의 예산은 사회개발, 사회보장, 사회복지 예산 뿐 아니라 문화공보과의 청소년관련예산, 도시정비과 예산, 건축 및 복권기금 등을 포함하고 있음. 그러므로 사회복지예산과 본 계획의 부문별 예산 합계는 약간의 차이가 있음. \n□ 신규사업의 경우 "지방재정법 시행령" 및 "보조금의 예산 및 관리에 관한 법률 시행령" 등에 준하여 분담비율 등을 산정하였음. \n□ 참여복지5개년계획 및 기타 보건복지관련 계획 및 지침 등을 고려하여 예산을 추정하였음. \n□ 향후 계획 수행의 편의를 위해 예산서의 예산과목을 최대한 고려하여 편성하였으나, 사업성격에 따라 예산과목을 통합하거 나 부문을 변경하였음. \n□ 예산 증가 추이 평가\n⚬ 전체 예산의 증가 추이는 2007년에는 전년대비 6.4%, 2008년 은 11.1%, 2009년은 -1.1%, 2010년에는 4.4%가 증가하고 평균 5.2%의 증가추이를 나타냄. \n⚬ 국내 각 연구기관에서 국내총생산 성장률을 4.0-5.0%로 예측 하고 있고, 노령화, 저출산 등과 관련하여 예산확보가 용이할 것으로 평가됨. \n⚬다만, 연차적으로 예산증가가 고르지 못한 것은 복지관련 시설 및 기관의 신축비 등이 포함되기 때문임. \n제5장 평가계획 및 평가의 한계\n제1절 평가계획\n1. 평가개요\n□ 지역사회복지계획은 사회복지사업법 제15조 6에서 보건복지부 장관 또는 시도지사가 기초자치단체의 지역사회복지계획의 시행결과를 평가할 수 있도록 규정. \n⚬ 지역사회복지계획은 2007년 1월에서 2010년 12월까지 실천되는 중기계획이므로 지역사회복지계획의 목표달성 여부를 확인하고 지역사회복지계획에 따라 지역사회복지 실천을 수정•보완하기 위해서는 이에 적합한 지역사회복지 평가계획 이 필요함. \n⚬ 지역사회복지계획은 향후 기초자치단체, 중앙정부 등에서 마련하고 있는 여타 지역계획 및 중앙계획가 보조를 맞추어 나가야 하므로 이를 검토하고 계획간 호환성 관련성을 제고 할 수 있도록 지역사회복지계획을 평가하여야 함. \n2. 평가방법 및 목적\n□ 평가목적\n⚬ 계양구의 지역사회복지계획 실천의 책임성 확보 및 효율적인 실천이 진행될 수 있도록 함. \n⚬ 평가를 통해 계양구 지역사회복지계획의 수정 및 보완을 하고 지역사회복지기반 확충 및 복지기관간 연계강화. \n⚬ 향후 지역사회복지계획이 4년마다 수립되어야 하므로 앞으로 계획 수립에 있어 실천적 계획 수립의 방법을 제시. \n□ 평가방법\n⚬ 사회복지사업법시행령 제7조 4(지역사회복지계획 시행결과의 평가)에 따르면 지역사회복지계획은 지역사회복지계획 내용의 충실성, 시행과정의 적정성, 시행결과의 목표달성도, 지역 주민의 참여도 및 만족도 등을 평가하도록 규정. \n⚬ 동조 2항에서는 지역사회복지계획 시행결과를 평가하기 위하여 기초자치단체 지역사회복지계획의 시행결과와 연차별 시행계획의 결과를 제출하도록 규정. \n⚬ 연간 실행계획에 따른 추진결과를 연 1회 자체 평가 실시하고 평가 시기는 매년 10월로 하여 예산반영 및 계획의 수정 및 검토. \n⚬ 지역사회복지계획이 수립되는 현 시점에서는 평가 영역만 명시되어 있어 영역별 배점 및 가중치, 영역별 평가항목 구성, 구체적인 평가방법 및 평가도구 개발에는 한계가 있음. \n⚬ 이러한 점들을 고려하여 지역사회복지계획 평가방법은 위의 4가지 영역을 중심으로 하여 평가방법을 제시함. \n□ 평가주체\n⚬ 평가의 주체는 계양구 지역사회복지계획을 수립•실천하는 지역사회가 주체가 되고 외부의 전문가가 참여하는 것이 평가의 공정성 및 책임성을 확보할 수 있음. \n⚬ 계양구 지역사회복지계획의 평가주체는 계양구, 계양구 사회 복지협의체, 계양구민, 외부전문가로 구성함. \n⚬ 계양구 지역사회복지계획의 평가를 위해 평가주체는 평가항목, 평가영역별 배점 및 가중치, 평가영역별 평가항목 구성, 구체적인 평가방법 등을 고려한 평가를 시행. \n제2절 평가활용방안 및 한계\n1. 평가활용방안\n□ 지역사회복지계획평가 활용방안\n⚬ 계양구 지역사회복지계획 실천의 우선순위 및 연차별 지역사회복지계획의 수정 및 보안. \n⚬ 계양구 우수사업의 사례관리 및 매뉴얼 작성을 통해 지역사회복지계획 달성. \n⚬ 평가결과를 반영하여 각종 계획 수립 및 지역사회발전을 위한 지역자원의 효율성 제고. \n2. 평가의 한계\n□ 자치구 사업으로서의 한계\n⚬ 계양구의 경우 재정자립도가 인천광역시 자치구 중 낮은 편에 속해있는 것이 사실이고, 그러므로 사업추진에 있어 국비 및 시비지원이 매우 필요함. \n⚬ 그러나 현실적으로 자치구의 사업계획에 따라 국비 및 시비 등이 모두 지원된다고 보기는 어려움. \n⚬ 또한, 중앙정부의 주요시책이 바뀔 경우 지방정부는 이에 저항 하기 어렵고, 그 중앙정부의 의지가 복지를 확대하는 방향이 아닐 경우 지방자치단체의 복지계획은 달성의 한계를 지님. \n□ 사회•경제적 불확실성으로 인한 한계\n⚬ 계획수립에 있어 재정확보를 경제성장률, 인플레이션, 지역경제 성장정도 등을 고려해야 하는데, 이는 예측이 매우 어려운 부분으로서 불확실한 미래상황을 제공. \n'
inputs_raw = f"""<|begin_of_text|><|start_header_id|>user: <|end_header_id|>Please summarize the following text to 20% of its length. There should be no spelling or typos, and the text should not be long or contain repeated similar content.
{inputs_raw}<|eot_id|><|start_header_id|>assistant: <|end_header_id|>
"""
# del inputs
torch.cuda.empty_cache()

# 입력과 종료 조건 설정
stop_words = ["."]  # 종료를 트리거하는 키워드
stopping_criteria = StoppingCriteriaList([StopOnKeyword(stop_words, tokenizer,len(inputs_raw)//100*25)])

inputs = tokenizer(inputs_raw, return_tensors="pt").to('cuda')

outputs = loaded_model.generate(
    input_ids=inputs["input_ids"], 
    attention_mask=inputs["attention_mask"],
    max_new_tokens=len(inputs_raw)//100*35,
    eos_token_id=128009,
    temperature=0.4,
    no_repeat_ngram_size=10,  # 반복을 방지
    # repetition_penalty = 1.1,
    stopping_criteria=stopping_criteria,  # 사용자 정의 종료 조건
    early_stopping=True,
    do_sample=True
    )



print(tokenizer.batch_decode(outputs, skip_special_tokens=False)[0].replace(inputs_raw, ''))

NameError: name 'loaded_model' is not defined

In [18]:
!export CUDA_LAUNCH_BLOCKING=1

UsageError: Cell magic `%%` not found.
